In [1]:
import numpy as np
import pandas as pd
import cloudpickle as pkl
import pandas_gbq
import torch
from google.cloud import bigquery
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from IPython.display import display

import base64
import re
import pandas as pd
from datetime import datetime
from time import sleep

client = bigquery.Client()

In [2]:
pd.set_option('display.max_columns', 500)

# PL Propensity Mixup

In [ ]:
QUERY = f"""
WITH 
RankedCustomers AS (
  SELECT
    internal.*,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY upload_date DESC) AS rn
  FROM
     `abcd-dataplatform.abcd_data_science_app.pl_explore_users_internal` AS internal
),
LatestRecords AS (
  SELECT
    rc.*
  FROM
    RankedCustomers AS rc
  WHERE
    rn = 1
)

SELECT
  lr.*
FROM
  LatestRecords lr
"""

test_conflict_set = client.query(QUERY).to_dataframe()
test_conflict_set.head()

In [ ]:
pandas_gbq.to_gbq(
    dataframe=test_conflict_set[test_conflict_set.sent==1],
    project_id='abcd-dataplatform',
    destination_table='abcd_data_science_app.test1',
    if_exists='replace'
)

In [ ]:
QUERY = f"""
WITH 
RankedCustomers AS (
  SELECT
    internal.*,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY upload_date DESC) AS rn
  FROM
     `abcd-dataplatform.abcd_data_science_app.pl_explore_users_internal` AS internal
),
LatestRecords AS (
  SELECT
    rc.*
  FROM
    RankedCustomers AS rc
  WHERE
    rn = 1
),
CustomerConflicts AS (
  SELECT
    final_records.*
  FROM
    LatestRecords as final_records
  WHERE
    customer_id NOT IN(
      SELECT
        prod_high.customer_id
      FROM
        `abcd-dataplatform-prod.abcd_analytics_ml_usecase_downstream_consumption.pl_explore_users_high_credit_score` AS prod_high
      WHERE
        upload_date IS NOT NULL
    )
    AND customer_id NOT IN (
      SELECT
        prod_miss.customer_id
      FROM
        `abcd-dataplatform-prod.abcd_analytics_ml_usecase_downstream_consumption.pl_explore_users_no_credit_score` AS prod_miss
      WHERE
        upload_date IS NOT NULL
    )
)

SELECT
  conflicts.*
FROM
  CustomerConflicts AS conflicts
"""

exclusion_set = client.query(QUERY).to_dataframe()
exclusion_set.head()

In [ ]:
pandas_gbq.to_gbq(
    dataframe=exclusion_set[exclusion_set.sent==1],
    project_id='abcd-dataplatform',
    destination_table='abcd_data_science_app.test2',
    if_exists='replace'
)

In [ ]:
exclusion_set.info()

In [ ]:
test_conflict_set.info()

In [ ]:
# Initialize the BigQuery client
client = bigquery.Client()
 
# Define the SQL query
query = """
WITH cte_base AS (
  SELECT
    *
  FROM
    `abcd-dataplatform.abcd_data_science_app.pl_explore_users_internal`
  WHERE
    sent = 1
),
cte_with_window_function AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY upload_date DESC) AS row_num
  FROM
    cte_base
)
SELECT
  *
FROM
  cte_with_window_function
WHERE
  row_num = 1
  AND EXPERIAN_Score_V3 >= 700
"""
 
# Executing the SQL query 
df_700 = client.query(query).to_dataframe().drop(columns=['group', 'decile', 'group_decile', 'sent', 'probability_score', 'row_num'])
 
df_700.columns

In [ ]:
df_700.customer_id.nunique(), df_700.shape[0]

In [ ]:
pandas_gbq.to_gbq(
    dataframe=df_700,
    project_id='abcd-dataplatform',
    destination_table='abcd_data_science_app.gt_700_snapshot',
    if_exists='replace'    
)

In [ ]:
# Initialize the BigQuery client
client = bigquery.Client()
 
# Define the SQL query
query = """
WITH cte_base AS (
  SELECT
    *
  FROM
    `abcd-dataplatform.abcd_data_science_app.pl_explore_users_internal`
  WHERE
    sent = 1
),
cte_with_window_function AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY upload_date DESC) AS row_num
  FROM
    cte_base
)
SELECT
  *
FROM
  cte_with_window_function
WHERE
  row_num = 1
  AND EXPERIAN_Score_V3 IS NULL
"""
 
# Executing the SQL query 
df_null = client.query(query).to_dataframe().drop(columns=['group', 'decile', 'group_decile', 'sent', 'probability_score', 'row_num'])
 
df_null.columns

In [ ]:
df_null.customer_id.nunique(), df_null.shape[0]

In [ ]:
pandas_gbq.to_gbq(
    dataframe=df_null,
    project_id='abcd-dataplatform',
    destination_table='abcd_data_science_app.null_snapshot',
    if_exists='replace'    
)

In [ ]:
QUERY = f"""
MERGE `abcd-dataplatform-prod.abcd_analytics_ml_usecase_downstream_consumption.pl_explore_users_high_credit_score` AS target
USING `abcd-dataplatform.abcd_data_science_app.gt_700_snapshot` AS source
ON target.customer_id = source.customer_id

WHEN MATCHED THEN
    UPDATE SET
        target.mobilenumber = source.mobilenumber,
        target.Status = source.Status,
        target.PA_FLAG = source.PA_FLAG,
        target.PQ_FLAG = source.PQ_FLAG,
        target.EXPERIAN_Score_V3 = source.EXPERIAN_Score_V3,
        target.firstname = source.firstname,
        target.lastname = source.lastname,
        target.PA_PQ_FLAG = source.PA_PQ_FLAG,
        target.upload_date = source.upload_date

WHEN NOT MATCHED THEN
    INSERT (
        customer_id, mobilenumber, Status, PA_FLAG, PQ_FLAG, 
        EXPERIAN_Score_V3, firstname, lastname, PA_PQ_FLAG, upload_date
    )
    VALUES (
        source.customer_id, source.mobilenumber, source.Status, 
        source.PA_FLAG, source.PQ_FLAG, source.EXPERIAN_Score_V3, 
        source.firstname, source.lastname, source.PA_PQ_FLAG, source.upload_date
    );
"""

In [ ]:
QUERY = f"""
MERGE `abcd-dataplatform-prod.abcd_analytics_ml_usecase_downstream_consumption.pl_explore_users_no_credit_score` AS target
USING `abcd-dataplatform.abcd_data_science_app.null_snapshot` AS source
ON target.customer_id = source.customer_id

WHEN MATCHED THEN
    UPDATE SET
        target.mobilenumber = source.mobilenumber,
        target.Status = source.Status,
        target.PA_FLAG = source.PA_FLAG,
        target.PQ_FLAG = source.PQ_FLAG,
        target.EXPERIAN_Score_V3 = source.EXPERIAN_Score_V3,
        target.firstname = source.firstname,
        target.lastname = source.lastname,
        target.PA_PQ_FLAG = source.PA_PQ_FLAG,
        target.upload_date = source.upload_date

WHEN NOT MATCHED THEN
    INSERT (
        customer_id, mobilenumber, Status, PA_FLAG, PQ_FLAG, 
        EXPERIAN_Score_V3, firstname, lastname, PA_PQ_FLAG, upload_date
    )
    VALUES (
        source.customer_id, source.mobilenumber, source.Status, 
        source.PA_FLAG, source.PQ_FLAG, source.EXPERIAN_Score_V3, 
        source.firstname, source.lastname, source.PA_PQ_FLAG, source.upload_date
    );
"""

# Distribution changes 

In [ ]:
others_list = [
    'staff', 
    'publisher', 
    'internal_assets', 
    'partner', 
    'internal_comms', 
    'branchcollateral', 
    'employee_referral', 
    'btl', 
    'brand', 
    'abg', 
    'social_media', 
    'atl'
]

others_query = ', '.join([f"'{ele}'" for ele in others_list])
QUERY = f"""
WITH RankedCustomers AS (
    SELECT
        a.customer_user_id AS customer_id,
        CASE
            WHEN b.top_channels IN ({others_query}) THEN 'others'
            ELSE b.top_channels
        END AS top_channels,
        ROW_NUMBER() OVER (PARTITION BY a.customer_user_id ORDER BY event_time DESC) AS rn
    FROM
        `abcd-dataplatform-prod.abcd_curated_zone.fact_appsflyer_metrics_updated` a
    LEFT JOIN
        `abcd-dataplatform.abcd_data_model.top_channels_july` b
    ON
        a.media_source=b.media_source
    WHERE
        customer_user_id is not null
)
SELECT
    customer_id,
    top_channels,
FROM
    RankedCustomers
WHERE
    rn = 1
"""

media_source_df = client.query(QUERY).to_dataframe()
media_source_df.info()

In [ ]:
media_source_df.customer_id.nunique()

In [ ]:
as_of_date = '2024-11-28'
# Perform a query.
###'abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer'
QUERY = f"""
WITH CTE_Demog AS (
    SELECT 
        A.id AS customer_id,A.mobilenumber As MOBILE,
        A.createddate as createdDate,
        A.isabc_employee as IS_ABC_EMPLOYEE,
        COALESCE(EXTRACT(YEAR FROM PARSE_DATE('%Y-%m-%d', '{as_of_date}')) - EXTRACT(YEAR FROM COALESCE(A.dateofbirth,G.dob, H.dob)), 
                 B.customer_age, 
                 EXTRACT(YEAR FROM PARSE_DATE('%Y-%m-%d', '{as_of_date}')) - EXTRACT(YEAR FROM PARSE_DATE('%Y-%m-%d', C.date_of_birth)),
                 E.age, 
                 F.customer_age, S.age) AS CUSTOMER_AGE_DEMOGS,
        COALESCE(J.annual_income, K.Annual_income, L.Annual_income, S.annual_income, 0) AS Annual_Income_demog,
        COALESCE(K.employment_type, L.employment_type) AS Employment_Type,
        COALESCE(M.pincode, CAST(N.pincode AS STRING), O.pincode, CAST(P.pincode AS STRING), CAST(Q.pincode AS STRING), CAST(R.pincode AS STRING)) AS PINCODE_DEMOGS,
        ROW_NUMBER() OVER (PARTITION BY A.id ORDER BY A.id) AS rn -- Modify the ORDER BY clause as needed
    FROM 
            `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer` AS A 
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_curated_zone.fact_customer_health_detail` AS B ON A.id = B.customer_id
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer_vas_recon` AS C ON A.id = C.customer_id
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_dha_data` AS E ON A.id = E.customer_id
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_curated_zone.dim_customer` AS F ON A.id = F.customer_id
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_homeloan_customer` AS G ON A.id = G.customer_id
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_hi_proposer_insured_members` AS H ON A.id = H.customer_id
    LEFT JOIN
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_li_basic_nomininee` AS I ON A.id = I.customer_id
    LEFT JOIN  
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer_professional_details` AS J ON A.id = J.customer_id
    LEFT JOIN
        (SELECT 
             customer_id,
             employment_type,
             SUM((monthly_income + 
                  IFNULL(other_income_amount, 0) +
                  IFNULL(other_income_amount_2, 0) +
                  IFNULL(other_income_amount_3, 0) +
                  IFNULL(other_income_amount_4, 0) +
                  IFNULL(other_income_amount_5, 0) +
                  IFNULL(other_income_amount_6, 0) +
                  IFNULL(other_income_amount_7, 0)
                 ) * 12) AS Annual_income
           FROM 
             `abcd-dataplatform-prod.abcd_curated_zone.fact_homeloan_employement_detail`
           GROUP BY 
             customer_id, employment_type
        ) AS K ON A.id = K.customer_id
    LEFT JOIN
        (SELECT
            customer_id,
            employment_type,
            SUM((monthly_income + 
                  IFNULL(other_income_amount, 0) +
                  IFNULL(other_income_amount2, 0) +
                  IFNULL(other_income_amount3, 0) +
                  IFNULL(other_income_amount4, 0) +
                  IFNULL(other_income_amount5, 0) +
                  IFNULL(other_income_amount6, 0) +
                  IFNULL(other_income_amount7, 0)
                 ) * 12) AS Annual_income
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_homeloan_employment_dtl`
            GROUP BY
              customer_id, employment_type
        ) AS L ON A.id = L.customer_id AND K.employment_type = L.employment_type
    LEFT JOIN
        (SELECT
            customerid,
            landmark,
            city,
            state,
            pincode
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_address`
        ) AS M ON A.id=M.customerid 
    LEFT JOIN
        (SELECT
            customer_id,
            MAX(city) AS city,
            MAX(state) AS state,
            MAX(pincode) AS pincode
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_homeloan_address` 
            GROUP BY
              customer_id
        ) AS N ON A.id=N.customer_id
    LEFT JOIN
        (SELECT
            customer_id,
            MAX(city) AS city,
            MAX(state) AS state,
            MAX(pincode) AS pincode
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_pl_address`
            GROUP BY
              customer_id
        ) AS O ON A.id=O.customer_id
    LEFT JOIN
        (SELECT
            customer_id,
            MAX(city) AS city,
            MAX(state) AS state,
            MAX(pin_code) AS pincode
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_bl_businessloan_details` 
            GROUP BY
              customer_id
        ) AS P ON A.id=CAST(P.customer_id AS INTEGER)
    LEFT JOIN
        (SELECT
            customer_id,
            MAX(city) AS city,
            MAX(state) AS state,
            MAX(pincode) AS pincode
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_mf_address_details`
            GROUP BY
              customer_id
        ) AS Q ON A.id=Q.customer_id
    LEFT JOIN
        (SELECT
            customer_id,
            MAX(kyc_landmark) AS landmark,
            MAX(kyc_city) AS city,
            MAX(kyc_state) AS state,
            MAX(kyc_pincode) AS pincode
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_li_address`
            GROUP BY
              customer_id
        ) AS R ON A.id=R.customer_id
    LEFT JOIN
        (SELECT
            customer_id,
            MAX(age) AS age,
            MAX(annual_income) AS annual_income,
            ANY_VALUE(gender) AS gender
            FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_li_quote_details` 
            GROUP BY
              customer_id
        ) AS S ON A.id=S.customer_id
)
SELECT
    CAST(customer_id AS STRING) customer_id,
    MOBILE,
    PINCODE_DEMOGS,
    CUSTOMER_AGE_DEMOGS
FROM CTE_Demog
WHERE rn = 1
"""

base_customer_df = client.query(QUERY).to_dataframe()
base_customer_df.info()

In [ ]:
base_customer_df.customer_id.nunique()

In [ ]:
media_source_df.shape, media_source_df.customer_id.nunique()

In [ ]:
customer_with_source = pd.merge(
    base_customer_df,
    media_source_df[['customer_id', 'top_channels']],
    on='customer_id',
    how='left'
)
customer_with_source.info()

In [ ]:
# customer_with_source.top_channels = customer_with_source.top_channels.fillna('unknown')
customer_with_source.top_channels.value_counts(dropna=False)

In [ ]:
customer_with_source.top_channels = customer_with_source.top_channels.fillna('unknown')
customer_with_source.top_channels.value_counts(dropna=False)

In [ ]:
segmentation_df = customer_with_source.copy()

In [ ]:
# Perform a query.
QUERY = (
    """
    WITH Appsflyer_records AS (
  SELECT
    A.appsflyerid,
    coalesce(cast(mp_user_id as int64), cast(guestid as int64)) as customer_id,
    A.time,
    B.platform, B.event_name, B.device_type, B.device_model, B.device_category, B.flag, B.app_version, B.city, B.state, B.postal_code, B.media_source, B.channel, B.language, B.operator, B.wifi, B.os_version, B.conversion_type,
    ROW_NUMBER() OVER (PARTITION BY A.appsflyerid ORDER BY A.time DESC) as row_num
  FROM
    `abcd-dataplatform-prod.abcd_mixpanel_raw.abcd_mp_master_event` A
  LEFT JOIN
    `abcd-dataplatform-prod.abcd_curated_zone.fact_appsflyer_metrics` B
  ON
    A.appsflyerid = B.appsflyer_id
)
SELECT
 distinct customer_id,
  appsflyerid,
  platform, event_name, device_type, device_model, device_category, flag AS ORGANIC_OR_INORGANIC_CONVERSION, app_version, city AS CITY_APPSFLYER, 
  state AS STATE_APPSFLYER, postal_code AS PINCODE_APPSFLYER, media_source, channel,language, operator, wifi, os_version, conversion_type,
  time
FROM
  Appsflyer_records
WHERE
  row_num = 1 and customer_id !=0 ;""")

query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish

# for row in rows:
#     print(row.name)

df_appsflyer = query_job.to_dataframe()

df_appsflyer.shape

In [ ]:
df_appsflyer['event_name'].value_counts(dropna = 0)

In [ ]:
df_appsflyer['event_name'].isnull().sum()

In [ ]:
df_appsflyer = df_appsflyer[df_appsflyer['event_name'].notna()]

df_appsflyer['event_name'].value_counts(dropna = 0)

In [ ]:
df_appsflyer = df_appsflyer.drop_duplicates(subset=['customer_id'], keep='first')

In [ ]:
df_appsflyer.shape

In [ ]:
df_appsflyer.head()

### VAS - Spend Manager, DHA, PC, Credit 

In [ ]:
QUERY=("""
select distinct id as customer_id,
SAVING_BALANCE AS SAVING_BALANCE_PC,
STOCK_BALANCE AS STOCK_BALANCE_PC ,
MF_Balance As MF_BALANCE_PC ,FD_DEPOSIT,
age As AGE_DHA , 
from `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer` ca
left join
(SELECT *
FROM
(SELECT tracking_id,current_balance AS SAVING_BALANCE, closing_balance as closing_balance_sb,
bank_name as bank_name_sb,
summary_status as summary_status_sb,
1 AS PC_Flag,
ROW_NUMBER() OVER (PARTITION BY tracking_id ORDER BY refresh_date DESC) AS rank
FROM `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_pc_bank_raw_summary`
) where rank=1) ba
on ca.id = ba.tracking_id
left join
(SELECT *
FROM
(SELECT tracking_id,total_deposit_value AS STOCK_BALANCE,
ROW_NUMBER() OVER (PARTITION BY tracking_id ORDER BY refresh_date DESC) AS rank
FROM `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_pc_bank_rawsummary_equities`
) where rank=1) eq
on ca.id = eq.tracking_id
left join
(select *
from
(SELECT bank_account_uuid,tracking_id,current_value AS MF_Balance,refresh_date,total_deposit_value as MF_Deposit_value,
ROW_NUMBER() OVER (PARTITION BY tracking_id ORDER BY refresh_date DESC) AS rank
FROM`abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_pc_bank_rawsummary_mf`)
where rank = 1) mf
on ca.id = mf.tracking_id
left join
(select *
from
(SELECT bank_account_uuid,tracking_id,total_deposit_value AS FD_Deposit,refresh_date,
ROW_NUMBER() OVER (PARTITION BY tracking_id ORDER BY refresh_date DESC) AS rank
FROM`abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_pc_bank_rawsummary_fd`)
where rank = 1) fd
on ca.id = fd.tracking_id
left join
(SELECT * 
FROM 
(SELECT customer_id,age,
ROW_NUMBER() over(partition by customer_id order by generated_at DESC )as rank
FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_dha_data` )
where rank=1) dha
on ca.id = dha.customer_id;
""")

In [ ]:
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish

# for row in rows:
#     print(row.name)

df_vas = query_job.to_dataframe()

df_vas.shape

In [ ]:
df_vas['customer_id'].nunique()

In [ ]:
df_vas.head()

In [ ]:
print(df_vas['customer_id'].nunique(),df_vas.shape)

### Spend Data

In [ ]:
# Perform a query.
QUERY = (
    """
SELECT 
  userId,
  category,
  SUM(CASE WHEN category = 'Travel' AND year = 2022 THEN totalAmount ELSE 0 END) AS Travel_amount_2022,
  SUM(CASE WHEN category = 'Travel' AND year = 2023 THEN totalAmount ELSE 0 END) AS Travel_amount_2023,
  SUM(CASE WHEN category = 'Travel' AND year = 2024 THEN totalAmount ELSE 0 END) AS Travel_amount_2024,
  SUM(CASE WHEN category = 'Loans/investing' AND year = 2022 THEN totalAmount ELSE 0 END) AS Loans_Investing_amount_2022,
  SUM(CASE WHEN category = 'Loans/investing' AND year = 2023 THEN totalAmount ELSE 0 END) AS Loans_Investing_amount_2023,
  SUM(CASE WHEN category = 'Loans/investing' AND year = 2024 THEN totalAmount ELSE 0 END) AS Loans_Investing_amount_2024,
  SUM(CASE WHEN category = 'Household/rent' AND year = 2022 THEN totalAmount ELSE 0 END) AS Rent_amount_2022,
  SUM(CASE WHEN category = 'Household/rent' AND year = 2023 THEN totalAmount ELSE 0 END) AS Rent_amount_2023,
  SUM(CASE WHEN category = 'Household/rent' AND year = 2024 THEN totalAmount ELSE 0 END) AS Rent_amount_2024,
  SUM(CASE WHEN category = 'Food' AND year = 2022 THEN totalAmount ELSE 0 END) AS Food_amount_2022,
  SUM(CASE WHEN category = 'Food' AND year = 2023 THEN totalAmount ELSE 0 END) AS Food_amount_2023,
  SUM(CASE WHEN category = 'Food' AND year = 2024 THEN totalAmount ELSE 0 END) AS Food_amount_2024,
  SUM(CASE WHEN category = 'Others' AND year = 2022 THEN totalAmount ELSE 0 END) AS Others_amount_2022,
  SUM(CASE WHEN category = 'Others' AND year = 2023 THEN totalAmount ELSE 0 END) AS Others_amount_2023,
  SUM(CASE WHEN category = 'Others' AND year = 2024 THEN totalAmount ELSE 0 END) AS Others_amount_2024,
  SUM(CASE WHEN category = 'Bills/Utilities' AND year = 2022 THEN totalAmount ELSE 0 END) AS utility_amount_2022,
  SUM(CASE WHEN category = 'Bills/Utilities' AND year = 2023 THEN totalAmount ELSE 0 END) AS utility_amount_2023,
  SUM(CASE WHEN category = 'Bills/Utilities' AND year = 2024 THEN totalAmount ELSE 0 END) AS utility_amount_2024,
  SUM(CASE WHEN category = 'Lifestyle' AND year = 2022 THEN totalAmount ELSE 0 END) AS Lifestyle_amount_2022,
  SUM(CASE WHEN category = 'Lifestyle' AND year = 2023 THEN totalAmount ELSE 0 END) AS Lifestyle_amount_2023,
  SUM(CASE WHEN category = 'Lifestyle' AND year = 2024 THEN totalAmount ELSE 0 END) AS Lifestyle_amount_2024
FROM (
  SELECT DISTINCT userId, category, year, totalAmount
  FROM `abcd-dataplatform-prod.abcd_spend_manager_transformed.mv_w5pt2_sm_yearly_spend_categories`
) AS distinct_data
GROUP BY userId, category;

 """)

In [ ]:
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish

# for row in rows:
#     print(row.name)

df_spend_data = query_job.to_dataframe()

df_spend_data.shape

In [ ]:
df_spend_data.head()

In [ ]:
df_spend_data['userId'].nunique()

In [ ]:
df_spend_data = df_spend_data[['userId','Travel_amount_2022', 'Travel_amount_2023',
       'Travel_amount_2024', 'Loans_Investing_amount_2022',
       'Loans_Investing_amount_2023', 'Loans_Investing_amount_2024',
       'Rent_amount_2022', 'Rent_amount_2023', 'Rent_amount_2024',
       'Food_amount_2022', 'Food_amount_2023', 'Food_amount_2024',
       'Others_amount_2022', 'Others_amount_2023', 'Others_amount_2024',
       'utility_amount_2022', 'utility_amount_2023', 'utility_amount_2024',
       'Lifestyle_amount_2022', 'Lifestyle_amount_2023',
       'Lifestyle_amount_2024']]

In [ ]:
df_spend_view = df_spend_data.groupby('userId').sum().reset_index()

In [ ]:
df_spend_view.columns

In [ ]:
df_spend_view.shape

In [ ]:
df_spend_view.head()

In [ ]:
df_spend_view['Total_Spend_2022'] = df_spend_view['Travel_amount_2022'].astype(float) + df_spend_view['Loans_Investing_amount_2022'].astype(float) + df_spend_view['Rent_amount_2022'].astype(float) + df_spend_view['Food_amount_2022'].astype(float) + df_spend_view['Others_amount_2022'].astype(float) + df_spend_view['utility_amount_2022'].astype(float) + df_spend_view['Lifestyle_amount_2022'].astype(float)
df_spend_view['Total_Spend_2023'] = df_spend_view['Travel_amount_2023'].astype(float) + df_spend_view['Loans_Investing_amount_2023'].astype(float) + df_spend_view['Rent_amount_2023'].astype(float) + df_spend_view['Food_amount_2023'].astype(float) + df_spend_view['Others_amount_2023'].astype(float) + df_spend_view['utility_amount_2023'].astype(float) + df_spend_view['Lifestyle_amount_2023'].astype(float)

df_spend_view['Avg_yearly_spend'] = (df_spend_view['Total_Spend_2022'].astype(float) + df_spend_view['Total_Spend_2023'].astype(float))/2

In [ ]:
df_spend_view=df_spend_view[['userId','Avg_yearly_spend']]

In [ ]:
QUERY=("""
Select channel_profile_id, count(distinct txn_id) AS NO_OF_UPI_TRANSACTION, sum(amount) AS TOTAL_GMV,1 as Payment_FLAG 
from abcd-dataplatform-prod.abcd_payments_system_raw.smpd_upi_common_smpd_common_fin_tranlog
where status in ("A","C") and created <= '2024-11-04' 
group by channel_profile_id
order by NO_OF_UPI_TRANSACTION desc;"""
)

query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish

# for row in rows:
#     print(row.name)

df_payment_data = query_job.to_dataframe()

df_payment_data.shape

### SIngle View

In [ ]:
df_appsflyer.customer_id = df_appsflyer.customer_id.astype(str)

In [ ]:
df_app_single_view = pd.merge(segmentation_df,df_appsflyer,on = 'customer_id',
                              how = 'left')

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view.customer_id = df_app_single_view.customer_id.astype('Int64')

In [ ]:
df_app_single_view = pd.merge(df_app_single_view,df_vas,on = 'customer_id',
                              how = 'left')

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['customer_id'] = df_app_single_view['customer_id'].astype(str)
df_spend_view['userId']=df_spend_view['userId'].astype(str)

In [ ]:
df_spend_view.shape,df_spend_view['userId'].nunique()

In [ ]:
df_app_single_view = pd.merge(df_app_single_view,df_spend_view,left_on = 'customer_id',right_on = 'userId',
                              how = 'left')

In [ ]:
df_app_single_view.shape

In [ ]:
print(df_payment_data['channel_profile_id'].nunique(),df_payment_data.shape)

In [ ]:
df_payment_data=df_payment_data.drop_duplicates(subset='channel_profile_id',keep='first')

print(df_payment_data['channel_profile_id'].nunique(),df_payment_data.shape)

In [ ]:
df_app_single_view = pd.merge(df_app_single_view,df_payment_data,left_on = 'customer_id',right_on = 'channel_profile_id',
                              how = 'left')

In [ ]:
df_app_single_view=df_app_single_view.drop(columns='channel_profile_id')

In [ ]:
df_app_single_view['Payment_FLAG']=df_app_single_view['Payment_FLAG'].fillna(0)

In [ ]:
df_app_single_view.shape

### MOBILE_PRICE_MAPPER

In [ ]:
#mobile_price_mapper = pd.read_excel("Mobile_Price_Mapper 4.xlsx")

In [ ]:
QUERY=(""" SELECT * FROM abcd-dataplatform.abcd_data_science_app.MOBILE_PRICE_MAPPER ;
""")

In [ ]:
query_job = client.query(QUERY)  # API request

mobile_price_mapper = query_job.to_dataframe()
mobile_price_mapper.shape

In [ ]:
mobile_price_mapper1 = mobile_price_mapper[['DEVICE_MODEL','Mobile_device_Price']]

In [ ]:
mobile_price_mapper2 = mobile_price_mapper[['MOBILE_Concate_Fields','Mobile_device_Price']]

In [ ]:
mobile_price_mapper1['DEVICE_MODEL'].nunique(),mobile_price_mapper.shape

In [ ]:
mobile_price_mapper1['DEVICE_MODEL']=mobile_price_mapper1['DEVICE_MODEL'].str.upper()
mobile_price_mapper2['MOBILE_Concate_Fields']=mobile_price_mapper2['MOBILE_Concate_Fields'].str.upper()

In [ ]:
mobile_price_mapper1['DEVICE_MODEL'].nunique(),mobile_price_mapper1.shape

In [ ]:
mobile_price_mapper1['DEVICE_MODEL'].nunique(),mobile_price_mapper1.shape

In [ ]:
mobile_price_mapper1 = mobile_price_mapper1.drop_duplicates(subset='DEVICE_MODEL')
mobile_price_mapper2 = mobile_price_mapper2.drop_duplicates(subset='MOBILE_Concate_Fields')

In [ ]:
df_app_single_view['device_model'].value_counts(dropna=0)

In [ ]:
df_app_single_view['DEVICE_MODEL']=df_app_single_view['device_model'].str.upper()

In [ ]:
mobile_price_mapper1['DEVICE_MODEL']=mobile_price_mapper1['DEVICE_MODEL'].str.upper()

In [ ]:
mobile_price_mapper2['MOBILE_Concate_Fields']=mobile_price_mapper2['MOBILE_Concate_Fields'].str.upper()

In [ ]:
df_app_single_view=pd.merge(df_app_single_view,mobile_price_mapper1,on='DEVICE_MODEL',how='left')

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view=pd.merge(df_app_single_view,mobile_price_mapper2,left_on='device_model',right_on='MOBILE_Concate_Fields',how='left')

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['Mobile_device_Price_x'].notna().sum()

In [ ]:
df_app_single_view['Mobile_device_Price_x'].value_counts(dropna=0)

In [ ]:
df_app_single_view['Mobile_device_Price_y'].value_counts(dropna=0)

In [ ]:
df_app_single_view.drop(columns=['Mobile_device_Price_y'], inplace=True)

In [ ]:
df_app_single_view = df_app_single_view.rename(columns={'Mobile_device_Price_x':'Mobile_device_Price'})

In [ ]:
df_app_single_view['Mobile_device_Price'].value_counts(dropna=0)

In [ ]:
df_app_single_view.rename(columns={'Mobile_device_Price': 'MOBILE_DEVICE_PRICE'}, inplace=True)

In [ ]:
df_app_single_view['MOBILE_DEVICE_PRICE'].value_counts(dropna=0)

In [ ]:
df_app_single_view['MOBILE_DEVICE_PRICE']=df_app_single_view['MOBILE_DEVICE_PRICE'].fillna('Unknown')

In [ ]:
df_app_single_view[df_app_single_view['MOBILE_DEVICE_PRICE']!='Unknown']['MOBILE_DEVICE_PRICE'].value_counts(normalize=1)*100

In [ ]:
100-73.639169-3.851897

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['MOBILE_DEVICE_PRICE'].value_counts(normalize=1)*100

In [ ]:
100-42.2-42.2-2.224857

### TIER MAPPER 

In [ ]:
df_app_single_view.filter(like='PINCODE').columns

In [ ]:
df_app_single_view['PINCODE_APPSFLYER'].notna().sum()

In [ ]:
df_app_single_view['FINAL_PINCODE']=np.where(df_app_single_view['PINCODE_APPSFLYER'].isnull(),df_app_single_view['PINCODE_DEMOGS'],df_app_single_view['PINCODE_APPSFLYER'])

In [ ]:
df_app_single_view['FINAL_PINCODE'].notna().sum()

In [ ]:
QUERY=(""" SELECT PINCODE,Tier FROM abcd-dataplatform.abcd_data_science_app.PINCODE_MAPPER;
""")

query_job = client.query(QUERY)  # API request

pincode_mapper = query_job.to_dataframe()
pincode_mapper.shape

In [ ]:
pincode_mapper['PINCODE']=pincode_mapper['PINCODE'].astype(str)

In [ ]:
df_app_single_view['FINAL_PINCODE']=df_app_single_view['FINAL_PINCODE'].astype(str)
pincode_mapper['PINCODE']=pincode_mapper['PINCODE'].apply(lambda x: x[:6])

df_app_single_view['FINAL_PINCODE']=df_app_single_view['FINAL_PINCODE'].apply(lambda x: x[:6]) 

In [ ]:
df_app_single_view['PINCODE_APPSFLYER'].notna().sum()

In [ ]:
df_app_single_view=pd.merge(df_app_single_view,pincode_mapper,left_on='FINAL_PINCODE',right_on='PINCODE',how='left')
 

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view=df_app_single_view.drop(columns='PINCODE')

In [ ]:
df_app_single_view['Tier'].value_counts(dropna=0)

In [ ]:
df_app_single_view.rename(columns={'Tier': 'TIER'}, inplace=True)

In [ ]:
df_app_single_view['TIER'].value_counts(dropna=0)

In [ ]:
#df_app_single_view.to_csv('app_data_single_view_updated_upto15thAUG_v1.csv',index=False)

##### SMS DATA 

In [ ]:
QUERY=("""

WITH user_financials AS (
  -- Aggregate income and expenses by user_id and year_month
  SELECT
    user_id,
    year_month,
    SUM(CAST(REPLACE(total_inflow, ',', '') AS FLOAT64)) AS Total_inflow,
    SUM(CAST(REPLACE(total_expense, ',', '') AS FLOAT64)) AS Total_expense
  FROM
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`
  GROUP BY
    user_id, year_month
),
 
bank_account_data AS (
  -- Aggregate bank account details by user_id and year_month
  SELECT
    user_id,
    year_month,
    MAX(latest_account.last_txn_date) AS last_txn_date,
    SUM(
      CASE
        WHEN SAFE_CAST(REPLACE(latest_account.balance, ',', '') AS FLOAT64) IS NOT NULL THEN
          SAFE_CAST(REPLACE(latest_account.balance, ',', '') AS FLOAT64)
        ELSE
          0
      END
    ) AS Total_balance,

    SUM(
      CASE
        WHEN SAFE_CAST(REPLACE(latest_account.monthly_credit, ',', '') AS FLOAT64) IS NOT NULL THEN
          SAFE_CAST(REPLACE(latest_account.monthly_credit, ',', '') AS FLOAT64)
        ELSE
          0
      END
    ) AS Total_monthly_credit, 
    SUM(
      CASE
        WHEN SAFE_CAST(REPLACE(latest_account.monthly_debit, ',', '') AS FLOAT64) IS NOT NULL THEN
          SAFE_CAST(REPLACE(latest_account.monthly_debit, ',', '') AS FLOAT64)
        ELSE
          0
      END
    ) AS Total_monthly_debit
  FROM (
    SELECT
      user_id,
      year_month,
      bank_account.*
    FROM
      `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`,
      UNNEST(balances.bank_accounts) AS bank_account
  ) AS latest_account
  GROUP BY
    user_id, year_month
),
 
investment_data AS (
  -- Aggregate investment details by user_id and year_month
  SELECT
    user_id,
    year_month,
    SUM(CAST(REPLACE(investment.amount, ',', '') AS FLOAT64)) AS Total_investment_amount
  FROM
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`,
    UNNEST(balances.investment) AS investment
  GROUP BY
    user_id, year_month
),
 
credit_card_data AS (
  -- Aggregate credit card details by user_id and year_month
  SELECT
    user_id,
    year_month,
    SUM(
      CASE
        WHEN SAFE_CAST(REPLACE(credit_card.available_limit, ',', '') AS FLOAT64) IS NOT NULL THEN
          SAFE_CAST(REPLACE(credit_card.available_limit, ',', '') AS FLOAT64)
        ELSE
          0
      END
    ) AS Total_credit_available_limit,
    SUM(
      CASE
        WHEN SAFE_CAST(REPLACE(credit_card.due_amount, ',', '') AS FLOAT64) IS NOT NULL THEN
          SAFE_CAST(REPLACE(credit_card.due_amount, ',', '') AS FLOAT64)
        ELSE
          0
      END
    ) AS Total_credit_due_amount,
 
    SUM(
      CASE
        WHEN SAFE_CAST(REPLACE(credit_card.total_credit_limit, ',', '') AS FLOAT64) IS NOT NULL THEN
          SAFE_CAST(REPLACE(credit_card.total_credit_limit, ',', '') AS FLOAT64)
        ELSE
          0
      END
    ) AS Total_credit_limit
  FROM
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`,
    UNNEST(balances.credit_card_accounts) AS credit_card
  GROUP BY
    user_id, year_month
),
 
loan_data AS (
  -- Aggregate loan details by user_id and year_month
  SELECT
    user_id,
    year_month,
    MAX(latest_loan.emi_date) AS latest_emi_date,
    MAX(latest_loan.emi_due_msg_date) AS latest_emi_due_msg_date,
    SUM(CAST(REPLACE(latest_loan.emi_due, ',', '') AS FLOAT64)) AS Total_emi_due,
 
    SUM(CAST(REPLACE(latest_loan.emi_paid, ',', '') AS FLOAT64)) AS Total_emi_paid
  FROM (
    SELECT
      user_id,
      year_month,
      loan.*
    FROM
      `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`,
      UNNEST(balances.loan_dues) AS loan
  ) AS latest_loan
  GROUP BY
    user_id, year_month
),
 
SMS_data AS (
  -- Combine all aggregated data by user_id and year_month
  SELECT
    uf.user_id,
    uf.year_month,
    uf.Total_inflow,
    uf.Total_expense,
    ba.last_txn_date,
    ba.Total_balance,
    ba.Total_monthly_credit,
    ba.Total_monthly_debit,
    inv.Total_investment_amount,
    cc.Total_credit_available_limit,
    cc.Total_credit_due_amount,
    cc.Total_credit_limit,
    ld.latest_emi_date,
    ld.latest_emi_due_msg_date,
    ld.Total_emi_due,
    ld.Total_emi_paid
  FROM
    user_financials uf
  LEFT JOIN
    bank_account_data ba ON uf.user_id = ba.user_id AND uf.year_month = ba.year_month
  LEFT JOIN
    investment_data inv ON uf.user_id = inv.user_id AND uf.year_month = inv.year_month
  LEFT JOIN
    credit_card_data cc ON uf.user_id = cc.user_id AND uf.year_month = cc.year_month
  LEFT JOIN
    loan_data ld ON uf.user_id = ld.user_id AND uf.year_month = ld.year_month
)

-- Calculate the final aggregated data by user_id
SELECT
  user_id,
  AVG(Total_inflow) AS AVG_MONTHLY_INFLOW,
  SUM(Total_inflow) AS SUM_inflow,
  MAX(Total_inflow) AS MAX_inflow,
   AVG(Total_expense) AS AVG_MONTHLY_EXPENSE,
   SUM(Total_expense) AS SUM_expense,
  MAX(Total_expense) AS MAX_expense,
  MAX(last_txn_date) AS last_txn_date,
  AVG(Total_balance) AS AVG_TOTAL_BALANCE,
  SUM(Total_balance) AS SUM_total_balance,
  MAX(Total_balance) AS MAX_total_balance,
  AVG(Total_monthly_credit) AS AVG_MONTHLY_CREDIT,  
  SUM(Total_monthly_credit) AS SUM_monthly_credit,
  MAX(Total_monthly_credit) AS MAX_monthly_credit,
  AVG(Total_monthly_debit) AS AVG_MONTHLY_DEBIT,   
  SUM(Total_monthly_debit) AS SUM_monthly_debit,
  MAX(Total_monthly_debit) AS MAX_monthly_debit,
  AVG(Total_investment_amount) AS AVG_INVESTMENT_AMOUNT,  
  SUM(Total_investment_amount) AS SUM_investment_amount,
  MAX(Total_investment_amount) AS MAX_investment_amount,
  AVG(Total_credit_available_limit) AS AVG_CREDIT_CARD_AVAILABLE_LIMIT,   
  SUM(Total_credit_available_limit) AS SUM_credit_available_limit,
  MAX(Total_credit_available_limit) AS MAX_credit_available_limit,
  AVG(Total_credit_due_amount) AS AVG_CREDIT_CARD_DUE_AMOUNT,   
  SUM(Total_credit_due_amount) AS SUM_credit_due_amount,
  MAX(Total_credit_due_amount) AS MAX_credit_due_amount,
  AVG(Total_credit_limit) AS AVG_TOTAL_CREDIT_LIMIT,   
  SUM(Total_credit_limit) AS SUM_total_credit_limit,
  MAX(Total_credit_limit) AS MAX_total_credit_limit,   
  MAX(latest_emi_date) AS latest_emi_date,
  MAX(latest_emi_due_msg_date) AS latest_emi_due_msg_date,
  AVG(Total_emi_due) AS AVG_MONTHLY_EMI_DUE,   
  SUM(Total_emi_due) AS SUM_emi_due,
  MAX(Total_emi_due) AS MAX_emi_due,
 AVG(Total_emi_paid) AS AVG_MONTHLY_EMI_PAID,
  SUM(Total_emi_paid) AS SUM_emi_paid,
  MAX(Total_emi_paid) AS MAX_emi_paid
FROM
  SMS_data
GROUP BY
  user_id ;

""")

In [ ]:
### Updated_SMS_QUERY_RUN _IN _USER_CASE

In [ ]:
query_job = client.query(QUERY)  # API request

df_SMS_data = query_job.to_dataframe()

df_SMS_data.shape

In [ ]:
df_SMS_data['user_id'].nunique()

In [ ]:
df_SMS_data['user_id']=df_SMS_data['user_id'].astype(str)
df_SMS_data['user_id']=df_SMS_data['user_id'].str[:10]

df_app_single_view['MOBILE']=df_app_single_view['MOBILE'].astype(str)
df_app_single_view['MOBILE']=df_app_single_view['MOBILE'].str[:10]

In [ ]:
df_app_single_view['MOBILE'] = df_app_single_view['MOBILE'].astype(str).str.strip()
df_SMS_data['user_id'] = df_SMS_data['user_id'].astype(str).str.strip()


In [ ]:
df_app_single_view=pd.merge(df_app_single_view,df_SMS_data,left_on='MOBILE',right_on='user_id',how='left')
 

In [ ]:
df_app_single_view=df_app_single_view.drop(columns='user_id')

In [ ]:
df_app_single_view.shape

In [ ]:
QUERY=("""
select appsflyer_id,a.media_source,a.customer_user_id customer_id,case when a.media_source is null or a.media_source="organic" then "Organic" else a.media_source end organic_unorganic,date(event_time) Date,b.top_channels as Top_channels,event_name
from `abcd-dataplatform-prod.abcd_curated_zone.fact_appsflyer_metrics_updated` a
left join `abcd-dataplatform.abcd_data_model.top_channels_july` b
on a.media_source=b.media_source
where customer_user_id is not null ;
""")

In [ ]:
query_job = client.query(QUERY)  # API request

df_top_channel_data = query_job.to_dataframe()

df_top_channel_data.shape


In [ ]:
df_top_channel_data.head()

In [ ]:
df_top_channel_data1=df_top_channel_data[['customer_id','Top_channels']]

In [ ]:
df_top_channel_data1['customer_id'].nunique(),df_top_channel_data1.shape

In [ ]:
df_top_channel_data1=df_top_channel_data1.drop_duplicates(subset='customer_id',keep='first')

In [ ]:
df_app_single_view['customer_id']=df_app_single_view['customer_id'].astype(str)

In [ ]:
df_top_channel_data1['customer_id']=df_top_channel_data1['customer_id'].astype(str)

In [ ]:
df_app_single_view['customer_id']=df_app_single_view['customer_id'].astype(str).str.strip()

In [ ]:
df_top_channel_data1['customer_id']=df_top_channel_data1['customer_id'].astype(str).str.strip()

In [ ]:
df_app_single_view=pd.merge(df_app_single_view,df_top_channel_data1,on='customer_id',how='left')


In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['Top_channels']=df_app_single_view['Top_channels'].str.upper()

In [ ]:
##df_app_single_view['top_channels'].value_counts(dropna=0)--top_channels is the origial 

In [ ]:
df_app_single_view['Top_channels'].value_counts(dropna=0)

In [ ]:
top_channels_mapper={'D2C':'D2C',
                    'PAID':'PAID',
                     #'ORGANIC':'ORGANIC',
                    'INTERNAL_ASSETS':'INTERNAL_ASSETS',
                     'EMPLOYEE_REFERRAL':'EMPLOYEE_REFERRAL',
                     'BTL':'BTL',
                     'AFFILIATE':'AFFILIATE',
                     'PAYMENTS':'PAYMENTS',
                     'BRANCHCOLLATERAL':'OTHER',
                     'INTERNAL_COMMS':'OTHER',
                     'BCSM':'OTHER',
                     'SOCIAL_MEDIA':'OTHER',
                     'PROGRAMMATIC':'OTHER'
                    }

df_app_single_view['Top_channels']=np.where(df_app_single_view['Top_channels'].isnull(),np.nan,df_app_single_view['Top_channels'].map(top_channels_mapper))

df_app_single_view['Top_channels']=df_app_single_view['Top_channels'].fillna('UNKNOWN')

In [ ]:
df_app_single_view['Top_channels'].value_counts(dropna=0)

In [ ]:
QUERY=("""
SELECT *  FROM abcd-dataplatform.abcd_data_science_app.EXPERIAN_DATA_APP_SEGMENTATIONS;
""")

In [ ]:
query_job = client.query(QUERY)  # API request

df_exp = query_job.to_dataframe()
 
df_exp.shape

In [ ]:
df_exp['CREDIT_FLAG'].value_counts(dropna=0)

In [ ]:
df_exp=df_exp.rename(columns={'CUSTOMER_ID':'MOBILE'})

In [ ]:
df_exp['MOBILE']=df_exp['MOBILE'].astype(str)
df_exp['MOBILE']=df_exp['MOBILE'].str[:10]

df_app_single_view['MOBILE']=df_app_single_view['MOBILE'].astype(str)
df_app_single_view['MOBILE']=df_app_single_view['MOBILE'].str[:10]

#SRIP

df_app_single_view['MOBILE'] = df_app_single_view['MOBILE'].astype(str).str.strip()
df_exp['MOBILE'] = df_exp['MOBILE'].astype(str).str.strip()

In [ ]:
df_app_single_view=pd.merge(df_app_single_view,df_exp,on='MOBILE',how='left')


In [ ]:
df_app_single_view['CREDIT_FLAG'].value_counts(dropna=0)

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['SCORE_V3'].notna().sum()

In [ ]:
df_app_single_view.shape

### Adhoc_TBLE

In [ ]:
QUERY=(""" WITH RankedScores AS (
    SELECT CUST_ID AS MOBILE,Score_V3,SCRUB_DATE,
           ROW_NUMBER() OVER (PARTITION BY CUST_ID ORDER BY SCRUB_DATE DESC) AS rn
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_SCORE_ADHOC_BASE`
)
SELECT *
FROM RankedScores
WHERE rn = 1;
 """)

In [ ]:
query_job = client.query(QUERY)  # API request
df_Experian_adhoc = query_job.to_dataframe()

df_Experian_adhoc.shape

In [ ]:
df_Experian_adhoc.head()

In [ ]:
df_adhoc1=df_Experian_adhoc[['MOBILE','Score_V3']]

In [ ]:
df_adhoc1['MOBILE'].nunique(),df_adhoc1.shape

In [ ]:
df_adhoc1=df_adhoc1.drop_duplicates(subset='MOBILE',keep='first')

In [ ]:
df_adhoc1=df_adhoc1.drop_duplicates()

In [ ]:
df_adhoc1['MOBILE'].nunique(),df_adhoc1.shape

In [ ]:
df_adhoc1['MOBILE']=df_adhoc1['MOBILE'].astype(str)
df_adhoc1['MOBILE']=df_adhoc1['MOBILE'].str[:10]

df_app_single_view['MOBILE']=df_app_single_view['MOBILE'].astype(str)
df_app_single_view['MOBILE']=df_app_single_view['MOBILE'].str[:10]

#SRIP

df_app_single_view['MOBILE'] = df_app_single_view['MOBILE'].astype(str).str.strip()
df_adhoc1['MOBILE'] = df_adhoc1['MOBILE'].astype(str).str.strip()

In [ ]:
df_app_single_view=pd.merge(df_app_single_view,df_adhoc1,on='MOBILE',how='left')


In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['customer_id'].nunique()

In [ ]:
df_app_single_view=df_app_single_view.drop_duplicates(subset='customer_id',keep='first')

In [ ]:
df_app_single_view.shape

In [ ]:
df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED']=np.where(df_app_single_view['SCORE_V3'].isnull(),df_app_single_view['Score_V3'],df_app_single_view['SCORE_V3'])

In [ ]:
df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED'].notna().sum()

In [ ]:
df_app_single_view['NTC_FLAG_Updated']=np.where(df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED'].isnull(),1,0)

In [ ]:
df_app_single_view['NTC_FLAG_Updated'].value_counts(dropna=0)

In [ ]:
df_app_single_view.columns.tolist()

In [ ]:
df_app_single_view=df_app_single_view.drop(columns=['AR_EXPERIAN_FLAG',
 'CUST_ID',
 #'SCORE_V3',
 #'CREDIT_FLAG',
 'Score_V3'])

In [ ]:
df_app_single_view['CREDIT_FLAG']=df_app_single_view['CREDIT_FLAG'].fillna(0)

In [ ]:
#### CREATING ENCODER 

In [ ]:
# Define the mapping of Top_channels to scores
score_mapping = {
    'UNKNOWN': 0.629020 ,
    'PAID':0.099476 ,
    'AFFILIATE': 0.015208,
    'D2C': 0.194964,
    'PAYMENTS': 0.010471,
    'INTERNAL_ASSETS': 0.023186,
    'EMPLOYEE_REFERRAL':0.013214,
    'OTHER':0.005984,
    'BTL': 0.008477
}

# Create Channel_Encoded_Low_Target4 based on Top_channels with handling of NaN
df_app_single_view['Channel_Encoded_Low_Target4'] = np.where(pd.isnull(df_app_single_view['top_channels']), 0, df_app_single_view['top_channels'].map(score_mapping))


In [ ]:
# Define the mapping of Top_channels to scores
score_mapping = {
    'UNKNOWN': 0.635674 ,
    'PAID':0.107617 ,
    'AFFILIATE': 0.004879,
    'D2C': 0.156140,
    'PAYMENTS': 0.010471,
    'INTERNAL_ASSETS': 0.036595,
    'EMPLOYEE_REFERRAL':0.026294,
    'OTHER':0.008132,
    'BTL':0.015451
}

# Create Channel_Encoded_Medium_Target4 based on Top_channels with handling of NaN
df_app_single_view['Channel_Encoded_Medium_Target4'] = np.where(pd.isnull(df_app_single_view['Top_channels']), 0, df_app_single_view['Top_channels'].map(score_mapping))


In [ ]:
# Define the mapping of Top_channels to scores
score_mapping = {
    'UNKNOWN': 0.644219 ,
    'PAID':0.088868 ,
    'AFFILIATE': 0.003075,
    'D2C': 0.165129,
    'PAYMENTS': 0.003690,
    'INTERNAL_ASSETS': 0.021525,
    'EMPLOYEE_REFERRAL':0.027368,
    'OTHER':0.006765,
    'BTL':0.039360
}

# Create Channel_Encoded_High_Target4 based on Top_channels with handling of NaN
df_app_single_view['Channel_Encoded_High_Target4'] = np.where(pd.isnull(df_app_single_view['Top_channels']), 0, df_app_single_view['Top_channels'].map(score_mapping))


#### Craeting FINAL AGE 

In [ ]:
def create_column_from_others(df, columns, new_column_name):
    """
    Create a new column in a DataFrame based on values from other columns.
    Parameters:
    df (pd.DataFrame): The DataFrame.
    columns (list): List of columns to use in order of precedence.
    new_column_name (str): Name of the new column to create.
    Returns:
    pd.DataFrame: DataFrame with the new column added.
    """
    if not columns or not isinstance(columns, list):
        raise ValueError("columns parameter must be a non-empty list of column names.")
    if not isinstance(new_column_name, str) or not new_column_name:
        raise ValueError("new_column_name must be a non-empty string.")
    # Initialize the new column with NaN values
    df[new_column_name] = np.nan
    # Iterate through the list of columns and fill new_column_name with the first non-null value
    for column in columns:
        if column in df.columns:
            df[new_column_name] = df[new_column_name].combine_first(df[column])
        else:
            raise ValueError(f"Column '{column}' does not exist in the DataFrame.")
    return df

In [ ]:
df_app_single_view.filter(like='_AGE').columns

In [ ]:
df_app_single_view['AGE'].notna().sum()

In [ ]:
age_columns=['AGE','CUSTOMER_AGE_DEMOGS']

In [ ]:
for col in age_columns:
    df_app_single_view[col] = df_app_single_view[col].replace(0, np.nan)


In [ ]:
df_app_single_view = create_column_from_others(df_app_single_view, age_columns, 'FINAL_AGE')

#### EXPERIAN_CW_BUCKET

In [ ]:
df_app_single_view.filter(like='DPD').columns

In [ ]:
df_app_single_view['OVERALL_DPD']=(df_app_single_view['MAX_DPD_IN_LAST_30_DAYS']+df_app_single_view['MAX_DPD_IN_LAST_90_DAYS']+
                                    df_app_single_view['MAX_DPD_IN_LAST_180_DAYS']+df_app_single_view['MAX_DPD_IN_LAST_360_DAYS']+
                                    df_app_single_view['MAX_DPD_IN_LAST_270_DAYS'])

In [ ]:
df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED'] = pd.to_numeric(
    df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED'], errors='coerce'
)

In [ ]:
cut_labels = ['0-650','651-675','676-700','701-750','750+']
cut_bins = [0,650,675,700,750,10000]
df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED_BINS'] = pd.cut(df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED'],bins = cut_bins, labels = cut_labels)

In [ ]:
cut_labels = ['0-1','2-3','4-5','6','6+']
cut_bins = [-1,1,3,5,6,100000]
df_app_single_view['DPD_ACCOUNTS_90DAYS_BINS'] = pd.cut(df_app_single_view['MAX_DPD_IN_LAST_90_DAYS'],bins = cut_bins, labels = cut_labels)

In [ ]:
cut_labels = ['0-1','2-3','4-5','6','6+']
cut_bins = [-1,1,3,5,6,100000]
df_app_single_view['OVERALL_DPD_BINS'] = pd.cut(df_app_single_view['OVERALL_DPD'],bins = cut_bins, labels = cut_labels)

In [ ]:
score_mapping = {
    '0-1': 5,
    '2-3': 4,
    '4-5': 3,
    '6': 2,
    '6+': 1
}
 
# Create EXPERIAN_CREDIT_SCORE_BINS_score based on EXPERIAN_CREDIT_SCORE_BINS with handling of NaN
df_app_single_view['DPD_ACCOUNTS_90DAYS_BINS_score'] = np.where(pd.isnull(df_app_single_view['DPD_ACCOUNTS_90DAYS_BINS']), 5, df_app_single_view['DPD_ACCOUNTS_90DAYS_BINS'].map(score_mapping))

In [ ]:
score_mapping = {
    '0-650': 0,
    '651-675': 2,
    '676-700': 3,
    '701-750': 4,
    '750+':5
}
# Create EXPERIAN_CREDIT_SCORE_BINS_score based on EXPERIAN_CREDIT_SCORE_BINS with handling of NaN
df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED_score'] = np.where(pd.isnull(df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED_BINS']), 0, df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED_BINS'].map(score_mapping))

In [ ]:
score_mapping = {
    '0-1': 5,
    '2-3': 4,
    '4-5': 3,
    '6': 2,
    '6+': 1
}
 
# Create EXPERIAN_CREDIT_SCORE_BINS_score based on EXPERIAN_CREDIT_SCORE_BINS with handling of NaN
df_app_single_view['OVERALL_DPD_BINS_score'] = np.where(pd.isnull(df_app_single_view['OVERALL_DPD_BINS']), 5, df_app_single_view['OVERALL_DPD_BINS'].map(score_mapping))

In [ ]:
df_app_single_view['SEGMENTATION_CW_SCORE_UPDATED']=(df_app_single_view['EXPERIAN_CREDIT_SCORE_UPDATED_score']+
                                              df_app_single_view['OVERALL_DPD_BINS_score']+
                                              df_app_single_view['DPD_ACCOUNTS_90DAYS_BINS_score'])

In [ ]:
cut_labels = ['LOW','MEDIUM','HIGH']
cut_bins = [-1,10,14,16]
df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] = pd.cut(df_app_single_view['SEGMENTATION_CW_SCORE_UPDATED'],bins = cut_bins, labels = cut_labels)

In [ ]:
df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'].value_counts(dropna=0,normalize=1)

### AFFLUENCE_SCORE

In [ ]:
df_app_single_view['PL_AMOUNT_EXP_OVERALL']=np.where(df_app_single_view['PL_AMOUNT_EXP']<=-1,-1, df_app_single_view['PL_AMOUNT_EXP'])
df_app_single_view['LAP_AMOUNT_EXP_OVERALL']=np.where(df_app_single_view['LAP_AMOUNT_EXP']<=-1,-1, df_app_single_view['LAP_AMOUNT_EXP'])
df_app_single_view['CREDIT_CARD_LIMIT_EXP_OVERALL']=np.where(df_app_single_view['CREDIT_CARD_BALANCE_EXP']<=-1,-1, df_app_single_view['CREDIT_CARD_BALANCE_EXP'])
df_app_single_view['BL_AMOUNT_EXP_OVERALL']=np.where(df_app_single_view['BL_AMOUNT_EXP']<=-1,-1, df_app_single_view['BL_AMOUNT_EXP'])
df_app_single_view['AL_AMOUNT_EXP_OVERALL']=np.where(df_app_single_view['AL_AMOUNT_EXP']<=-1,-1, df_app_single_view['AL_AMOUNT_EXP'])
df_app_single_view['HL_AMOUNT_EXP_OVERALL']=np.where(df_app_single_view['HL_AMOUNT_EXP_SUM_ACTIVE']<=-1,-1, df_app_single_view['HL_AMOUNT_EXP_SUM_ACTIVE'])

In [ ]:
df_app_single_view['TIER']=df_app_single_view['TIER'].apply(lambda x: 1 if x=='Tier1' else(2 if x=='Tier2' else(3 if x=='Tier3' else np.nan)))

In [ ]:
df_app_single_view['TIER'].value_counts(dropna=0)

In [ ]:
replace_by_zero_list=['PL_AMOUNT_EXP_OVERALL',
 'BL_AMOUNT_EXP_OVERALL',
 'HL_AMOUNT_EXP_OVERALL',
 'AL_AMOUNT_EXP_OVERALL',
 #'HL_Amount_EXP_SUM_ACTIVE',
 'LAP_AMOUNT_EXP_OVERALL',
 'CREDIT_CARD_LIMIT_EXP_OVERALL',
 #'Mobile_device_Price',
'TIER','FINAL_AGE',
 #'HI_SUMINSURED',
 #'AMC_AUM',
 'TOTAL_GMV',
#'Total_LI_Sum_Insured',
'STOCK_BALANCE_PC','MF_BALANCE_PC','FD_DEPOSIT','SAVING_BALANCE_PC','Avg_yearly_spend']

#Replace by zero
for col in replace_by_zero_list:
    df_app_single_view[col]=df_app_single_view[col].fillna(0)
    

In [ ]:
# Define the mapping of MOBILE_DEVICE_PRICE to scores
score_mapping = {
    '0': 0,
    'Unknown':0,
    '< 10000': 1,
    '10000 to 25000': 2,
    '25000 to 50000': 3,
    '50000 to 100000': 4,
    '100000+':5
}

# Create MOBILE_DEVICE_PRICE_SCORE based on EXPERIAN_CREDIT_SCORE_BINS with handling of NaN
df_app_single_view['Mobile_device_Price'] = np.where(pd.isnull(df_app_single_view['MOBILE_DEVICE_PRICE']), 0, df_app_single_view['MOBILE_DEVICE_PRICE'].map(score_mapping))


In [ ]:
####

In [ ]:
df_app_single_view['AVG_TOTAL_CREDIT_LIMIT']

In [ ]:
data_model=df_app_single_view[['customer_id','AL_AMOUNT_EXP_OVERALL', 'Avg_yearly_spend',
       'BL_AMOUNT_EXP_OVERALL', 'CREDIT_CARD_LIMIT_EXP_OVERALL',
       'Channel_Encoded_Low_Target4', 'Channel_Encoded_Medium_Target4',
       'Channel_Encoded_High_Target4', 'FD_DEPOSIT', 'FINAL_AGE',
       'HL_AMOUNT_EXP_OVERALL', 'LAP_AMOUNT_EXP_OVERALL', 'MF_BALANCE_PC',
       'Mobile_device_Price', 'PL_AMOUNT_EXP_OVERALL',
       'SAVING_BALANCE_PC', 'STOCK_BALANCE_PC', 'TIER', 'TOTAL_GMV',
       'AVG_CREDIT_CARD_AVAILABLE_LIMIT', 'AVG_CREDIT_CARD_DUE_AMOUNT',
       'AVG_MONTHLY_EMI_DUE', 'AVG_MONTHLY_EMI_PAID', 'AVG_MONTHLY_EXPENSE', 'AVG_MONTHLY_INFLOW',
       'AVG_INVESTMENT_AMOUNT', 'AVG_MONTHLY_CREDIT', 'AVG_MONTHLY_DEBIT',
       'AVG_TOTAL_BALANCE', 'AVG_TOTAL_CREDIT_LIMIT']]

In [ ]:
columns=['AVG_CREDIT_CARD_AVAILABLE_LIMIT','AVG_CREDIT_CARD_DUE_AMOUNT','AVG_MONTHLY_EMI_DUE',
 'AVG_MONTHLY_EMI_PAID','AVG_MONTHLY_EXPENSE','AVG_MONTHLY_INFLOW','AVG_INVESTMENT_AMOUNT','AVG_MONTHLY_CREDIT','AVG_MONTHLY_DEBIT',
 'AVG_TOTAL_BALANCE','AVG_TOTAL_CREDIT_LIMIT']

replace_by_columns=['AVG_credit_available_limit', 'AVG_credit_due_amount',
       'AVG_emi_due', 'AVG_emi_paid', 'AVG_expense', 'AVG_inflow',
       'AVG_investment_amount', 'AVG_monthly_credit', 'AVG_monthly_debit',
       'AVG_total_balance', 'AVG_total_credit_limit']

# Create a mapping dictionary
rename_mapping = dict(zip(columns, replace_by_columns))

In [ ]:
rename_mapping

In [ ]:
# Rename columns in the DataFrame
data_model.rename(columns=rename_mapping, inplace=True)

In [ ]:
data_model.columns.tolist()

### LOAD_PICKEL FILE 

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

with open(r"Model_Target4_W_SMS_score.pkl", 'rb') as f:
    loaded_model = pkl.load(f)


In [ ]:
# Define XGBoost classifier
xgb = XGBClassifier(colsample_bytree=1.0, eval_metric='mlogloss', max_depth=5,
              num_class=3, objective='multi:softprob',learning_rate= 0.1,
              random_state=42, subsample=0.8, use_label_encoder=False)

In [ ]:
loaded_model.feature_names_in_

In [ ]:
data_model['TOTAL_GMV'] = pd.to_numeric(data_model['TOTAL_GMV'], errors='coerce')

In [ ]:
data_model1=data_model[['AL_AMOUNT_EXP_OVERALL', 'Avg_yearly_spend',
       'BL_AMOUNT_EXP_OVERALL', 'CREDIT_CARD_LIMIT_EXP_OVERALL',
       'Channel_Encoded_Low_Target4', 'Channel_Encoded_Medium_Target4',
       'Channel_Encoded_High_Target4', 'FD_DEPOSIT', 'FINAL_AGE',
       'HL_AMOUNT_EXP_OVERALL', 'LAP_AMOUNT_EXP_OVERALL', 'MF_BALANCE_PC',
       'Mobile_device_Price', 'PL_AMOUNT_EXP_OVERALL',
       'SAVING_BALANCE_PC', 'STOCK_BALANCE_PC', 'TIER', 'TOTAL_GMV',
       'AVG_credit_available_limit', 'AVG_credit_due_amount',
       'AVG_emi_due', 'AVG_emi_paid', 'AVG_expense', 'AVG_inflow',
       'AVG_investment_amount', 'AVG_monthly_credit', 'AVG_monthly_debit',
       'AVG_total_balance', 'AVG_total_credit_limit']]

In [ ]:
data_model['Predict_results'] = loaded_model.predict(data_model1)

In [ ]:
data_model['Predict_results'].value_counts(dropna=0)

In [ ]:
# Define the mapping of MOBILE_DEVICE_PRICE to scores
score_mapping = {
    2: 'HIGH',
    1:'MEDIUM',
    0: 'LOW'
}

# Create MOBILE_DEVICE_PRICE_SCORE based on EXPERIAN_CREDIT_SCORE_BINS with handling of NaN
data_model['Affluence_Score_Updated'] = np.where(pd.isnull(data_model['Predict_results']), 0, data_model['Predict_results'].map(score_mapping))


data_model['Affluence_Score_Updated'].value_counts(dropna=0,normalize=1)

In [ ]:
df_app_single_view['customer_id'].dtype

data_model['customer_id'].dtype

df_app_single_view=pd.merge(df_app_single_view,data_model[['customer_id','Affluence_Score_Updated']],on='customer_id',how='left')

df_app_single_view['Affluence_Score_Updated'].value_counts(dropna=0)

In [ ]:
# Assuming data_single_view_for_pivot is your DataFrame
conditions = [
    (df_app_single_view['Affluence_Score_Updated'] == 'HIGH') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'HIGH'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'MEDIUM') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'HIGH'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'LOW') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'HIGH'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'HIGH') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'MEDIUM'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'MEDIUM') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'MEDIUM'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'LOW') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'MEDIUM'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'HIGH') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'LOW'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'MEDIUM') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'LOW'),
    
    (df_app_single_view['Affluence_Score_Updated'] == 'LOW') & 
    (df_app_single_view['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'] == 'LOW')
]

# Corresponding values for the conditions
values = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# Create the new 'category' column
df_app_single_view['SEGMENT_Category'] = np.select(conditions, values, default=np.nan)


In [ ]:
df_app_single_view.SEGMENT_Category.value_counts(dropna=False).sort_index()

In [ ]:
drop_columns=[ 'Mobile_device_Price','Channel_Encoded_Low_Target4',
 'Channel_Encoded_Medium_Target4',
 'Channel_Encoded_High_Target4', 'Top_channels', 'MOBILE_Concate_Fields','userId',
 'rn',
 'appsflyerid',
 'platform',
 'event_name',
 'device_type',
 'device_model',
 'device_category']

In [ ]:
df_app_single_view=df_app_single_view.drop(columns=drop_columns)

In [ ]:
# #data=pd.read_csv('ABCDSA-3400 CE-status DECLINED.csv')

# data['id']=data['id'].astype(str)

# data_join=df_app_single_view[['customer_id','SEGMENTATION_CW_SCORE_BUCKET_UPDATED','Affluence_Score_Updated']] 

# data_join['customer_id']=data_join['customer_id'].astype(str)

# data1=pd.merge(data,data_join,left_on='id',right_on='customer_id',how='left')

# data1['Affluence_Score_Updated'].value_counts(dropna=0)

# data1['Affluence_Score_Updated']=data1['Affluence_Score_Updated'].fillna('LOW')

# data1['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'].value_counts(dropna=0)

# data1['SEGMENTATION_CW_SCORE_BUCKET_UPDATED']=data1['SEGMENTATION_CW_SCORE_BUCKET_UPDATED'].fillna('LOW')

# data1.shape

# data.shape

# data1.head(1)

# data1.to_csv('ABCDSA-3400 CE-status DECLINED_V1.csv',index=False)

In [ ]:
pandas_gbq.to_gbq(df_app_single_view, "abcd_data_science_app.ABCD_Segmentation_AFF_CW_FLAG" , 
                  project_id = "abcd-dataplatform", if_exists = "replace")

# Vintage Testing

In [ ]:
QUERY = f"""
SELECT
    *
FROM
    abcd-dataplatform.abcd_data_science_app.digigold_vintage_test
"""

python_calc = client.query(QUERY).to_dataframe()

In [ ]:
QUERY = f"""
SELECT
    *
FROM
    abcd-dataplatform.abcd_data_science_app.digigold_vintage_query_test
"""

query_calc = client.query(QUERY).to_dataframe()

In [ ]:
python_calc.shape, query_calc.shape, python_calc.customer_id.nunique(), query_calc.customer_id.nunique()

In [ ]:
query_extra = query_calc[~(query_calc.customer_id.isin(python_calc.customer_id))]
query_extra.shape

In [ ]:
python_extra = python_calc[~(python_calc.customer_id.isin(query_calc.customer_id))]
python_extra.shape

In [ ]:
query_extra.createddate

# Experian Analysis

## Fresh draw

In [ ]:
column_list = [
    "CUSTOMER_ID AS mobilenumber", 
    "ACCT_KEY",
    "ACCT_TYPE_CD", 
    "SAFE_CAST(DAYS_PAST_DUE_01 AS FLOAT64) AS DAYS_PAST_DUE_01",
    "SAFE_CAST(DAYS_PAST_DUE_02 AS FLOAT64) AS DAYS_PAST_DUE_02",
    "SAFE_CAST(DAYS_PAST_DUE_03 AS FLOAT64) AS DAYS_PAST_DUE_03",
    "SAFE_CAST(DAYS_PAST_DUE_04 AS FLOAT64) AS DAYS_PAST_DUE_04",
    "SAFE_CAST(DAYS_PAST_DUE_05 AS FLOAT64) AS DAYS_PAST_DUE_05",
    "SAFE_CAST(DAYS_PAST_DUE_06 AS FLOAT64) AS DAYS_PAST_DUE_06",
    "SAFE_CAST(DAYS_PAST_DUE_07 AS FLOAT64) AS DAYS_PAST_DUE_07",
    "SAFE_CAST(DAYS_PAST_DUE_08 AS FLOAT64) AS DAYS_PAST_DUE_08",
    "SAFE_CAST(DAYS_PAST_DUE_09 AS FLOAT64) AS DAYS_PAST_DUE_09",
    "SAFE_CAST(DAYS_PAST_DUE_10 AS FLOAT64) AS DAYS_PAST_DUE_10",
    "SAFE_CAST(DAYS_PAST_DUE_11 AS FLOAT64) AS DAYS_PAST_DUE_11",
    "SAFE_CAST(DAYS_PAST_DUE_12 AS FLOAT64) AS DAYS_PAST_DUE_12",
    "SAFE.PARSE_DATE('%d/%m/%Y', OPEN_DT) AS OPEN_DT", 
    "SAFE_CAST(BALANCE_AM AS FLOAT64) AS BALANCE_AM", 
    "SAFE.PARSE_DATE('%d/%m/%Y', BALANCE_DT) AS BALANCE_DT", 
    "SAFE.PARSE_DATE('%d/%m/%Y', CLOSED_DT) AS CLOSED_DT", 
    "SAFE_CAST(CREDIT_LIMIT_AM AS FLOAT64) AS CREDIT_LIMIT_AM", 
    "SAFE_CAST(ORIG_LOAN_AM AS FLOAT64) AS ORIG_LOAN_AM",
    "SCRUB_DATE"
]
columns_query = ',\n'.join(column_list)

In [ ]:
updated_column_list = [
    "mobilenumber", 
    "ACCT_KEY", 
    "ACCT_TYPE_CD", 
    "DAYS_PAST_DUE_01",
    "DAYS_PAST_DUE_02",
    "DAYS_PAST_DUE_03",
    "DAYS_PAST_DUE_04",
    "DAYS_PAST_DUE_05",
    "DAYS_PAST_DUE_06",
    "DAYS_PAST_DUE_07",
    "DAYS_PAST_DUE_08",
    "DAYS_PAST_DUE_09",
    "DAYS_PAST_DUE_10",
    "DAYS_PAST_DUE_11",
    "DAYS_PAST_DUE_12",
    "OPEN_DT",
    "BALANCE_AM", 
    "BALANCE_DT", 
    "CLOSED_DT", 
    "CREDIT_LIMIT_AM", 
    "ORIG_LOAN_AM",
    "SCRUB_DATE"
]
updated_columns_query = ', \n'.join(updated_column_list)

##### Kaustav's Idea

In [ ]:
QUERY = f"""
WITH commen_customers AS (
    SELECT
        CUSTOMER_ID AS mobilenumber,
        SAFE.PARSE_DATE('%d/%m/%Y', future.BALANCE_DT) AS future_balance_date,
        SAFE.PARSE_DATE('%d/%m/%Y', past.BALANCE_DT) AS past_balance_date,
        future.SCRUB_DATE AS future_scrub_date,
        past.SCRUB_DATE AS past_scrub_date
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED` AS future
    JOIN
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED` AS past
    ON
        future.CUSTOMER_ID = past.CUSTOMER_ID
        AND future.SCRUB_DATE = DATE_ADD(past.SCRUB_DATE, INTERVAL 30 DAY)
    WHERE
        future.SCRUB_DATE = '2024-12-31'
)
SELECT
    mobilenumber,
    future_balance_date.
"""

test_customer_df = client.query(QUERY).to_dataframe()
test_customer_df.info()

In [ ]:
QUERY = f"""
SELECT
    t1.customer_id,
    t1.ACCT_KEY,
    t1.ACCT_TYPE_CD,
    t1.DAYS_PAST_DUE_01,
    t1.DAYS_PAST_DUE_02,
    t1.DAYS_PAST_DUE_03,
    t1.DAYS_PAST_DUE_04,
    t1.DAYS_PAST_DUE_05,
    t1.DAYS_PAST_DUE_06,
    t1.DAYS_PAST_DUE_07,
    t1.DAYS_PAST_DUE_08,
    t1.DAYS_PAST_DUE_09,
    t1.DAYS_PAST_DUE_10,
    t1.DAYS_PAST_DUE_11,
    t1.DAYS_PAST_DUE_12,
    t1.OPEN_DT,
    t1.BALANCE_AM,
    t1.BALANCE_DT,
    t1.CLOSED_DT,
    t1.CREDIT_LIMIT_AM,
    t1.ORIG_LOAN_AM,
    t1.SCRUB_DATE,
    t2.ACCT_TYPE_CD,
    t2.DAYS_PAST_DUE_01,
    t2.DAYS_PAST_DUE_02,
    t2.DAYS_PAST_DUE_03,
    t2.DAYS_PAST_DUE_04,
    t2.DAYS_PAST_DUE_05,
    t2.DAYS_PAST_DUE_06,
    t2.DAYS_PAST_DUE_07,
    t2.DAYS_PAST_DUE_08,
    t2.DAYS_PAST_DUE_09,
    t2.DAYS_PAST_DUE_10,
    t2.DAYS_PAST_DUE_11,
    t2.DAYS_PAST_DUE_12,
    t2.OPEN_DT,
    t2.BALANCE_AM,
    t2.BALANCE_DT,
    t2.CLOSED_DT,
    t2.CREDIT_LIMIT_AM,
    t2.ORIG_LOAN_AM,
    t2.SCRUB_DATE
FROM (
    SELECT
        customer_id,
        ACCT_KEY,
        ACCT_TYPE_CD,
        DAYS_PAST_DUE_01,
        DAYS_PAST_DUE_02,
        DAYS_PAST_DUE_03,
        DAYS_PAST_DUE_04,
        DAYS_PAST_DUE_05,
        DAYS_PAST_DUE_06,
        DAYS_PAST_DUE_07,
        DAYS_PAST_DUE_08,
        DAYS_PAST_DUE_09,
        DAYS_PAST_DUE_10,
        DAYS_PAST_DUE_11,
        DAYS_PAST_DUE_12,
        OPEN_DT,
        BALANCE_AM,
        BALANCE_DT,
        CLOSED_DT,
        CREDIT_LIMIT_AM,
        ORIG_LOAN_AM,
        SCRUB_DATE
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
    WHERE 
        scrub_date >= '2024-11-23' 
        AND scrub_date <= '2024-12-07'
    ) AS t1
inner join (
    SELECT
        customer_id,
        ACCT_KEY,
        ACCT_TYPE_CD,
        DAYS_PAST_DUE_01,
        DAYS_PAST_DUE_02,
        DAYS_PAST_DUE_03,
        DAYS_PAST_DUE_04,
        DAYS_PAST_DUE_05,
        DAYS_PAST_DUE_06,
        DAYS_PAST_DUE_07,
        DAYS_PAST_DUE_08,
        DAYS_PAST_DUE_09,
        DAYS_PAST_DUE_10,
        DAYS_PAST_DUE_11,
        DAYS_PAST_DUE_12,
        OPEN_DT,
        BALANCE_AM,
        BALANCE_DT,
        CLOSED_DT,
        CREDIT_LIMIT_AM,
        ORIG_LOAN_AM,
        SCRUB_DATE
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
    WHERE 
        scrub_date > '2024-12-07'
    ) AS t2
ON 
    t1.customer_id = t2.customer_id 
    AND t1.acct_key = t2.acct_key
WHERE 
    t1.balance_dt <> t2.balance_dt
"""

test_customer_df = client.query(QUERY).to_dataframe()
test_customer_df.info()

In [ ]:
test_customer_df.head().to_csv('balance_dt_diff.csv', index=False)

In [ ]:
QUERY = f"""
WITH BaseData AS (
  SELECT DISTINCT ACCT_KEY
  FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
  WHERE SCRUB_DATE = '2024-11-23'
)

SELECT
  SCRUB_DATE,
  COUNT(ACCT_KEY),
FROM
  `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
WHERE
  ACCT_KEY IN (SELECT ACCT_KEY FROM BaseData)
  AND SCRUB_DATE > '2024-11-23'
GROUP BY 
  SCRUB_DATE
ORDER BY 
  SCRUB_DATE
"""

test_customer_df = client.query(QUERY).to_dataframe()
test_customer_df.info()

##### My Idea

In [ ]:
QUERY = f"""
WITH LatestData AS (
    SELECT
        RANK() OVER (PARTITION BY mobilenumber, ACCT_KEY ORDER BY source ASC, SCRUB_DATE DESC) AS rank,
        {updated_columns_query}
    FROM (
        SELECT
            {columns_query},
            1 AS source
        FROM
            `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
        
        UNION ALL
        
        SELECT
            {columns_query},
            0 AS source
        FROM
            `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_ADHOC_BASE_UPDATED`
        WHERE
            REMARKS = 'APP_SIGN_UP'
    )
    QUALIFY rank = 1
),

QualityData AS (
    SELECT
        mobilenumber,
        ACCT_KEY,
        ACCT_TYPE_CD,
        DAYS_PAST_DUE_01,
        DAYS_PAST_DUE_02,
        DAYS_PAST_DUE_03,
        DAYS_PAST_DUE_04,
        DAYS_PAST_DUE_05,
        DAYS_PAST_DUE_06,
        DAYS_PAST_DUE_07,
        DAYS_PAST_DUE_08,
        DAYS_PAST_DUE_09,
        DAYS_PAST_DUE_10,
        DAYS_PAST_DUE_11,
        DAYS_PAST_DUE_12,
        CASE 
            WHEN OPEN_DT < DATE('1900-01-01') THEN NULL
            ELSE OPEN_DT
        END AS OPEN_DT,
        BALANCE_DT,
        CASE 
            WHEN CLOSED_DT < DATE('1899-12-30') THEN NULL
            ELSE CLOSED_DT
        END AS CLOSED_DT,
        CREDIT_LIMIT_AM, 
        ORIG_LOAN_AM AS ORIG_ORIG_LOAN_AM,
        CASE
            WHEN ORIG_LOAN_AM < 0 THEN NULL
            ELSE ORIG_LOAN_AM
        END AS ORIG_LOAN_AM,
        BALANCE_AM AS ORIG_BALANCE_AM,
        CASE
            WHEN BALANCE_AM < 0 THEN NULL
            ELSE BALANCE_AM
        END AS BALANCE_AM,
        SCRUB_DATE
    FROM
        LatestData
)

SELECT
    *
FROM
    QualityData
"""

In [ ]:
job_config = bigquery.QueryJobConfig(dry_run=True)
exp_qj = client.query(QUERY, job_config=job_config)

while exp_qj.state != "DONE":
    exp_qj.reload()  # Refresh the job state
    print(f"Job Status: {exp_qj.state}")
    print(f"Total Bytes Processed: {exp_qj.total_bytes_processed or 'N/A'}")
    print(f"Total Slot Time (ms): {exp_qj.slot_millis or 'N/A'}")
    sleep(2)  # Poll every 2 seconds

# # Retrieve final job details once complete
# if exp_qj.state == "DONE" and not exp_qj.error_result:
#     print("Query Plan:")
#     for stage in exp_qj.statistics.query['queryPlan']:
#         print(f"Stage: {stage['name']}")
#         print(f"  Input Rows: {stage['recordsRead']}")
#         print(f"  Output Rows: {stage['recordsWritten']}")
#         print(f"  Compute Time (ms): {stage['computeMsAvg']}")

if exp_qj.error_result:
    print("Query failed with error:", exp_qj.error_result)
else:
    print(f"Query completed successfully. Total Size of Data Processed: {exp_qj.total_bytes_processed / (1024 ** 3):.2f} GB")
    print(f"Query Results URL: {exp_qj.self_link}")
# Get number of rows and estimated size

In [ ]:
exp_qj = client.query(QUERY)
exp_latest_scrub = exp_qj.to_dataframe()
exp_latest_scrub.info()

In [ ]:
exp_latest_scrub.ORIG_BALANCE_AM.value_counts(dropna=False)

In [ ]:
exp_latest_scrub[exp_latest_scrub.ORIG_BALANCE_AM < -1]

In [ ]:
exp_latest_scrub.to_csv('latest_scrub_daily_base_2025-09-01.csv', index=False)

## Load from Old

In [ ]:
exp_latest_scrub = pd.read_csv('latest_scrub_daily_base_2024-12-05.csv', low_memory=False)

In [ ]:
exp_latest_scrub.mobilenumber.isna().any()

In [ ]:
exp_latest_scrub.SCRUB_DATE.value_counts(dropna=False).sort_index()

In [ ]:
test_group = exp_latest_scrub.groupby(['mobilenumber', 'ACCT_KEY'])
test_group.ngroups

In [ ]:
exp_latest_scrub.shape

In [ ]:
pd.reset_option('display.max_rows')
dpd_value_counts = exp_latest_scrub[
    exp_latest_scrub.DAYS_PAST_DUE_03 == 900
][['mobilenumber', 'DAYS_PAST_DUE_03']].value_counts()

dpd_value_counts[dpd_value_counts > 1]

In [ ]:
test_customer = exp_latest_scrub[
    (exp_latest_scrub.mobilenumber == '9345899585')
]
test_customer.ACCT_KEY.value_counts()

In [ ]:
test_customer.to_csv('test_customer_for_experian_analysis.csv', index=False)

In [ ]:
test_customer = pd.read_csv('test_customer_for_experian_analysis.csv', low_memory=False)

In [ ]:
test_customer.ACCT_KEY.value_counts()

In [ ]:
QUERY = f"""
SELECT
    table_name,
    schema
FROM
    `abcd-dataplatform-prod.abcd_digitap_transformed.INFORMATION_SCHEMA.TABLES`
WHERE
    table_name = 'users_digitap_sms_data_transformed'
"""

test_columns = client.query(QUERY).to_dataframe()

In [ ]:
test_columns

In [ ]:
table_id = 'abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed'
table = client.get_table(table_id)

In [ ]:
def print_schema(fields, prefix=""):
    for field in fields:
        print(f"{prefix}{field.name}: {field.field_type}")
        if field.fields:  # Recursively print nested fields
            print_schema(field.fields, prefix=prefix + "  ")

print_schema(table.schema)

In [ ]:
for field in table.schema:
    if field.field_type == 'RECORD':
        print(field.name)
        print()

In [ ]:
balance_list = [
    'balances.bank_accounts.balance',
    'balances.bank_accounts.monthly_credit',
    'balances.bank_accounts.monthly_dedit',
    'balances.bank_accounts.bank',
    'balances.bank_accounts.account',
    'balances.bank_accounts.last_txn_date',
]

balance_query = ', '.join(balance_list)
balance_query

In [ ]:
balances: RECORD
  bank_accounts: RECORD
    balance: STRING
    monthly_credit: STRING
    monthly_debit: STRING
    bank: STRING
    account: STRING
    last_txn_date: STRING

In [ ]:
def schema_to_dict(fields):
    schema_dict = []
    for field in fields:
        field_dict = {
            "name": field.name,
            "type": field.field_type,
            "mode": field.mode,
        }
        if field.fields:  # Check if the field has nested fields
            field_dict["fields"] = schema_to_dict(field.fields)  # Recursively handle nested fields
        schema_dict.append(field_dict)
    return schema_dict

schema_dict = schema_to_dict(table.schema)

In [ ]:
schema_dict

In [ ]:
def generate_field_names_from_schema(schema_dict, prefix=""):
    result = []
    for field in schema_dict:  # Iterate over each field in the schema
        field_name = f"{prefix}{field['name']}"
        if field['type'] == 'RECORD':  # If the field is nested
            nested_fields = generate_field_names_from_schema(field['fields'], prefix=f"{field_name}.")
            result.extend(nested_fields)
        else:  # If it's a simple field
            result.append(field_name)
    return result

field_names = generate_field_names_from_schema(schema_dict)

In [ ]:
field_names

In [ ]:
QUERY = f"""
SELECT
  customer_id,
  Affluence_Score_Updated,
  NTC_FLAG_Updated
FROM
  `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE` AS cs
JOIN
  (
    SELECT
      DISTINCT user_id
    FROM
      `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`
  ) AS sms
ON
  cs.MOBILE = sms.user_id
"""

df = client.query(QUERY).to_dataframe()
df.info()

In [ ]:
df.NTC_FLAG_Updated.value_counts(dropna=False)

In [ ]:
QUERY = f"""
WITH NTC_users AS (
    SELECT
      customer_id,
      MOBILE,
      NTC_FLAG_Updated
    FROM
      `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`
    WHERE
        NTC_FLAG_Updated = 1
),
filtered_users AS (
    SELECT DISTINCT mdn AS user_id
    FROM `abcd-dataplatform.abcd_data_science_app.app_present_kb`
    WHERE mdn IN (
        SELECT
            MOBILE
        FROM
            NTC_Users
    )
),
mutual_fund as(select distinct a.user_id
from `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` a,
unnest(balances.investment) as i
where i.type in ('Mutual Fund','MF REDEMPTION','MF EXPENSE','SHARE & TRADING') and
amount='0.00')
select count(distinct a.user_id) from
`abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` a
JOIN filtered_users f ON a.user_id = f.user_id
JOIN mutual_fund m ON a.user_id = m.user_id;

"""

df_final = client.query(QUERY).to_dataframe()
df_final

# Mixpanel Exploration

In [ ]:
ng_data = pd.read_csv('Nikhil PL Exploration Data.csv', low_memory=False)
ng_data['time'] = pd.to_datetime(ng_data['time']).dt.tz_convert('Asia/Kolkata')

In [ ]:
ng_data = ng_data.sort_values('time', ascending=False)

In [ ]:
filtered = ng_data[ng_data.time.dt.date == pd.to_datetime('2024-12-13').date()]

In [ ]:
ng_data.info()

In [ ]:
unique_events = filtered.mp_event_name.unique()
len(unique_events)

In [ ]:
QUERY = f"""
SELECT
  *
FROM
  `abcd-dataplatform.abcd_data_science_app.MixPanel_Analysis_View_13thDEC` AS mp
WHERE
  DATE(DATETIME(mp.time, "Asia/Kolkata")) = "2024-12-13" 
"""

testclient.query(QUERY)

In [ ]:
QUERY = f"""
SELECT
  *
FROM
  `abcd-dataplatform.abcd_data_science_app.MixPanel_Analysis_View_13thDEC` AS mp
"""

In [ ]:
job_config = bigquery.QueryJobConfig(dry_run=True)
exp_qj = client.query(QUERY, job_config=job_config)

while exp_qj.state != "DONE":
    exp_qj.reload()  # Refresh the job state
    print(f"Job Status: {exp_qj.state}")
    print(f"Total Bytes Processed: {exp_qj.total_bytes_processed or 'N/A'}")
    print(f"Total Slot Time (ms): {exp_qj.slot_millis or 'N/A'}")
    sleep(2)  # Poll every 2 seconds

# # Retrieve final job details once complete
# if exp_qj.state == "DONE" and not exp_qj.error_result:
#     print("Query Plan:")
#     for stage in exp_qj.statistics.query['queryPlan']:
#         print(f"Stage: {stage['name']}")
#         print(f"  Input Rows: {stage['recordsRead']}")
#         print(f"  Output Rows: {stage['recordsWritten']}")
#         print(f"  Compute Time (ms): {stage['computeMsAvg']}")

if exp_qj.error_result:
    print("Query failed with error:", exp_qj.error_result)
else:
    print(f"Query completed successfully. Total Size of Data Processed: {exp_qj.total_bytes_processed / (1024 ** 3):.2f} GB")
    print(f"Query Results URL: {exp_qj.self_link}")
# Get number of rows and estimated size

In [ ]:
mp_qj = client.query(QUERY)
mp_view = mp_qj.to_dataframe()
mp_view.info()

In [ ]:
QUERY = f"""
SELECT
    *
FROM
    `abcd-dataplatform.abcd_data_science_app.INFORMATION_SCHEMA.COLUMNS`
WHERE
    table_name = 'MixPanel_Analysis_View_13thDEC'
"""
mixpanel_columns = client.query(QUERY).to_dataframe()
mixpanel_columns.info()

In [ ]:
# Table details
project_id = "abcd-dataplatform"
dataset_id = "abcd_data_science_app"
table_id = "MixPanel_Analysis_View_13thDEC"

# Step 2: Get the list of all column names
table_ref = f"{project_id}.{dataset_id}.{table_id}"
table = client.get_table(table_ref)

column_names = [schema_field.name for schema_field in table.schema]

# Step 3: Build query to identify columns with non-NULL values
non_null_columns = []

for col in column_names:
    QUERY = f"""
        SELECT COUNTIF({col} IS NOT NULL) AS non_null_count
        FROM `{table_ref}`
    """
    result = client.query(QUERY).result()
    non_null_count = [row['non_null_count'] for row in result][0]

    if non_null_count > 0:  # Keep columns with at least one non-NULL value
        non_null_columns.append(col)

In [ ]:
print(QUERY)

In [ ]:
# Step 4: Construct and execute the final query
if non_null_columns:
    final_query = f"""
        SELECT {', '.join(non_null_columns)}
        FROM `{table_ref}`
    """
    final_result = client.query(final_query).to_dataframe()
    print(final_result.info())
else:
    print("No columns with non-NULL values found.")

In [ ]:
mixpanel_columns.to_csv('MixPanel_Column_Schemas.csv', index=False)

# SMS data

In [ ]:
aff_data = pd.read_csv('Income_data.csv', low_memory=False)

In [ ]:
aff_data = aff_data.drop(columns='Unnamed: 0')
aff_data.head()

In [ ]:
aff_data.info()

In [ ]:
QUERY = f"""
SELECT 
    DISTINCT user_id AS MOBILE, 
    year_month,
    total_salary_credit,
    ROW_NUMBER() OVER (PARTITION BY user_id, year_month ORDER BY year_month DESC) AS row_num
FROM 
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`
WHERE
    total_salary_credit IS NOT NULL
QUALIFY
    row_num = 1
"""

monthly_salary_credit = client.query(QUERY).to_dataframe()
monthly_salary_credit.info()

In [ ]:
monthly_salary_credit['MOBILE'] = monthly_salary_credit['MOBILE'].astype(str)
aff_data['MOBILE'] = aff_data['MOBILE'].astype(str)

In [ ]:
monthly_salary_credit.MOBILE.nunique()

In [ ]:
monthly_salary_credit.info()

In [ ]:
group_counts = monthly_salary_credit.groupby(['MOBILE', 'year_month']).ngroups

In [ ]:
group_counts, monthly_salary_credit.shape[0] - group_counts

In [ ]:
merged_data = pd.merge(
    aff_data,
    monthly_salary_credit[['MOBILE', 'year_month', 'total_salary_credit']],
    on='MOBILE',
    how='left'
)
merged_data.info()

In [ ]:
merged_data.to_csv('Merged_Income_Data_Draft.csv', index=False)

In [ ]:
sms_hits = merged_data[merged_data.total_salary_credit.astype('Float64') > 0]
sms_hits.head()

In [ ]:
sms_hits = sms_hits.reset_index(drop=True)
sms_hits

In [ ]:
sms_hits.customer_id.nunique()

In [ ]:
base_data = sms_hits.groupby(['MOBILE']).agg({
    'INCOME_FOR_MODEL_PL_HFL' : lambda x: x.mean() / 12,
    'total_salary_credit' : lambda x: x.mode()[0]
})
base_data

In [ ]:
base_data = base_data.reset_index()
base_data

In [ ]:
base_data['total_salary_credit'] = base_data['total_salary_credit'].astype('Float64')

In [ ]:
base_data['RMSE'] = (np.sqrt((base_data['total_salary_credit'] - base_data['INCOME_FOR_MODEL_PL_HFL']) ** 2) * 100 / base_data['INCOME_FOR_MODEL_PL_HFL'])

base_data

In [ ]:
bins = [0, 5, 10, 20, 30, 50, float('inf')]  # Bin edges
labels = ['0-5%', '5-10%', '10-20%', '20-30%', '30-50%', '>50%']  # Bin labels

base_data['RMSE Bin'] = pd.cut(base_data['RMSE'], bins=bins, labels=labels, right=False)
base_data

In [ ]:
overall_variance = pd.concat([base_data.total_salary_credit, base_data.INCOME_FOR_MODEL_PL_HFL])
overall_variance = overall_variance.var()
overall_variance

In [ ]:
base_data['RMSE Bin'].value_counts()

In [ ]:
sms_hits['total_salary_credit'] = sms_hits['total_salary_credit'].astype('Float64')
sms_hits

In [ ]:
sms_hits.INCOME_FOR_MODEL_PL_HFL = sms_hits.INCOME_FOR_MODEL_PL_HFL / 12
sms_hits

In [ ]:
sms_hits['Squared Error'] = (sms_hits.INCOME_FOR_MODEL_PL_HFL - sms_hits.total_salary_credit) ** 2
sms_hits

In [ ]:
final_report = sms_hits.groupby(['MOBILE']).agg({
    'INCOME_FOR_MODEL_PL_HFL' : 'mean',
    'Squared Error' : lambda x: np.sqrt(x.mean())
}).reset_index()
final_report

In [ ]:
final_report = final_report.rename(columns={'Squared Error': 'RMSE'})
final_report

In [ ]:
final_report['RMSE %'] = final_report.RMSE * 100 / final_report.INCOME_FOR_MODEL_PL_HFL
final_report

In [ ]:
bins = [0, 5, 10, 20, 30, 50, float('inf')]  # Bin edges
labels = ['0-5%', '5-10%', '10-20%', '20-30%', '30-50%', '>50%']  # Bin labels

final_report['RMSE Bin'] = pd.cut(final_report['RMSE %'], bins=bins, labels=labels, right=False)
final_report

In [ ]:
final_report['RMSE Bin'].value_counts()

In [ ]:
base_data[base_data['RMSE Bin'] == '>50%']

In [ ]:
monthly_salary_credit.to_csv('SMS_salary_data.csv', index=False)

In [ ]:
aff_data.to_csv('LoB_Income_data.csv', index=False)

In [ ]:
%%bigquery  df

SELECT user_id, body FROM `abcd-dataplatform-prod.abcd_digitap_transformed.users_sms_narration_data_transformed` WHERE time = '2024-10-31'
and ingestion_time BETWEEN DATETIME("2024-10-31") AND DATETIME_ADD("2024-11-01", INTERVAL 1 DAY)

In [ ]:
%%bigquery  df

SELECT 
  user_id,
  body 
FROM 
  `abcd-dataplatform-prod.abcd_digitap_transformed.users_sms_narration_data_transformed` 
WHERE 
  ingestion_time >= "2024-10-31" 
  AND ingestion_time =< "2024-11-02"

In [ ]:
df.info(memory_usage='deep')

# Homeloan table

In [ ]:
%%bigquery loan_info_data

SELECT
    *
FROM
    `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_homeloan_customer` AS cust
LEFT JOIN
    `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_homeloan_loanoffer` AS offer
ON
    cust.customer_id = offer.customer_id
LEFT JOIN
    `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_homeloan_address` AS address
ON
    cust.customer_id = address.customer_id
LEFT JOIN
    `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_homeloan_address` AS empl
ON
    cust.customer_id = empl.customer_id

In [ ]:
loan_info_data

In [ ]:
loan_info_data.groupby('customer_id').count()

In [ ]:
loan_info_data[loan_info_data.customer_id == 17294].to_csv('Home_Loan_customer_table.csv', index=False)

In [ ]:
loan_info_data.info()

# Digitap NLP exercise

In [ ]:
%%bigquery raw_sms

WITH SMS_Data AS (
    SELECT 
        DISTINCT user_id, 
        year_month,
        total_salary_credit,
        ROW_NUMBER() OVER (PARTITION BY user_id, year_month ORDER BY year_month DESC) AS row_num
    FROM 
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`
    WHERE
        total_salary_credit IS NOT NULL
    QUALIFY
        row_num = 1
)
SELECT 
    user_id,
    device_id,
    time,
    sms_from,
    ingestion_time,
    body 
FROM 
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_sms_narration_data_transformed`
WHERE 
    ingestion_time >= "2024-10-31" 
    AND ingestion_time <= "2024-11-02"
    AND REGEXP_CONTAINS(body, r'^[A-Za-z0-9\s.,!?\'"()\-\:;]*$')
    AND user_id IN (
        SELECT 
            DISTINCT user_id
        FROM 
            SMS_Data
    )

In [ ]:
raw_sms.info(memory_usage='deep')

In [ ]:
fill_rate = raw_sms.notnull().mean() * 100  # Multiply by 100 for percentage
print(fill_rate)

In [ ]:
def encode_to_base64(text):
    if isinstance(text, str):  # Ensure the input is a string
        encoded_bytes = base64.b64encode(text.encode("utf-8"))  # Convert to bytes, then encode
        encoded_text = encoded_bytes.decode("utf-8")  # Convert bytes back to string
        return encoded_text
    else:
        return None  # Return None for invalid or missing input

In [ ]:
raw_sms['body'] = raw_sms['body'].apply(encode_to_base64)

In [ ]:
raw_sms['user_id'] = raw_sms['user_id'].apply(encode_to_base64)

In [ ]:
raw_sms

In [ ]:
raw_sms.to_csv('Raw_SMS_Data.csv', index=False)

In [ ]:
raw_sms_read = pd.read_csv('Raw_SMS_Data.csv', low_memory=False)

In [ ]:
def decode_from_base64(encoded_text):
    if isinstance(encoded_text, str):  # Ensure the input is a string
        try:
            decoded_bytes = base64.b64decode(encoded_text)  # Decode to bytes
            decoded_text = decoded_bytes.decode("utf-8")  # Convert bytes back to string
            return decoded_text
        except Exception as e:
            print(f"Decoding failed: {e}")
            return None  # Return None or a placeholder for decoding failure
    else:
        return None  # Return None if the input is not a string (e.g., float or NaN)

In [ ]:
raw_sms_read['body'] = raw_sms_read['body'].apply(decode_from_base64)

In [ ]:
raw_sms_read.info(memory_usage='deep')

In [ ]:
raw_sms_read

In [ ]:
raw_sms_read['body'] = raw_sms_read['body'].apply(encode_to_base64)

In [ ]:
raw_sms_read.head(500000).to_csv('Raw_SMS_Data_Sample.csv', index=False)

In [ ]:
raw_sms_read['user_id'] = raw_sms_read['user_id'].apply(decode_from_base64)
raw_sms_read

In [ ]:
raw_sms_read.columns

In [ ]:
raw_sms_read.shape

In [ ]:
filtered_data = raw_sms_read[raw_sms_read.body.notna()]
filtered_data.shape

In [ ]:
credited_filter = filtered_data[filtered_data.body.str.contains('credited')]
credited_filter

In [ ]:
def extract_money(text):
    # Regular expression to match monetary values
    pattern = r'(\$[0-9,]+(?:\.\d{1,2})?|€[0-9,]+(?:\.\d{1,2})?|₹[0-9,]+(?:\.\d{1,2})?|USD[0-9,]+(?:\.\d{1,2})?|EUR[0-9,]+(?:\.\d{1,2})?|Rs\.\s?[0-9,]+(?:\.\d{1,2})?|Rs\s?[0-9,]+(?:\.\d{1,2})?|INR\s?[0-9,]+(?:\.\d{1,2})?)'
    matches = re.findall(pattern, text)
    return matches if matches else None

In [ ]:
credited_filter['monetary_amount'] = credited_filter.body.apply(extract_money)
credited_filter

In [ ]:
odd_rows = credited_filter[credited_filter.monetary_amount.isna()]

In [ ]:
odd_rows.body

In [ ]:
def extract_numeric_amounts(monetary_values):
    if not monetary_values:
        return []
    numeric_values = []
    for value in monetary_values:
        # Remove currency symbols and commas, then convert to float
        cleaned_value = re.sub(r'[^\d.]', '', value)
        try:
            numeric_values.append(float(cleaned_value))
        except ValueError:
            pass
    return numeric_values

In [ ]:
credited_filter['flag_greater_than_10000'] = credited_filter['monetary_amount'].apply(
    lambda x: any(amount > 10000 for amount in extract_numeric_amounts(x)) if x else False
)

In [ ]:
credited_filter[credited_filter.flag_greater_than_10000]

In [ ]:
credited_filter['value_len'] = credited_filter.monetary_amount.apply(lambda x: len(x) if x is not None else 0)
credited_filter

In [ ]:
credited_filter[
    (credited_filter.value_len > 1)
]

In [ ]:
credited_filter['body'] = credited_filter['body'].apply(encode_to_base64)
credited_filter

In [ ]:
credited_filter[credited_filter.value_len > 1].to_csv('Odd_Customers_SMS.csv', index=False)

In [ ]:
credited_filter.to_csv('Extracted_SMS_Data.csv', index=False)

In [ ]:
credited_filter['body'] = credited_filter['body'].apply(decode_from_base64)
credited_filter

In [ ]:
credited_filter[credited_filter.value_len > 1].iloc[1].body

In [ ]:
with pd.option_context('display.max_colwidth', 2000):
    display(credited_filter.to_html())

In [ ]:
odd_rows = credited_filter[
    (credited_filter.flag_greater_than_10000)
    & (credited_filter.value_len == 1)
    & (credited_filter.body.str.contains('debit'))
]

In [ ]:
odd_rows.iloc[4].body

In [ ]:
# Load the model and tokenizer manually
model_name = "facebook/bart-large-mnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to perform zero-shot classification (detect salary data)
def detect_salary_data(text):
    # Define candidate labels for classification
    candidate_labels = ["Salary Data", "No Salary Data"]
    
    # Tokenize the input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    
    # Pass the inputs through the model
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Calculate probabilities (softmax over logits)
    scores = logits.softmax(dim=-1).squeeze().tolist()

    # Get the predicted label (highest score)
    predicted_class_idx = torch.argmax(logits, dim=-1).item()
    
    # Return results
    return {
        "labels": candidate_labels,
        "scores": scores,
        "predicted_class": candidate_labels[predicted_class_idx]
    }

In [ ]:
credited_filter['model_salary_prediction'] = credited_filter.body.apply(lambda x: detect_salary_data(x)['predicted_class'])

In [ ]:
# Load the model and tokenizer
model_name = "facebook/bart-large-mnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sample text input for checking the output shape
text = "This is a test sentence."

# Tokenize the input text
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

# Pass the inputs through the model
with torch.no_grad():
    outputs = model(**inputs)

# Check the output logits
logits = outputs.logits

# Print the shape of the logits
print("Logits shape:", logits.shape)

In [ ]:
# Initialize the zero-shot-classification pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# Define the candidate labels for salary detection
candidate_labels = ["Salary Data", "No Salary Data"]

# Example text to classify
text = "The candidate was offered a salary of Rs. 15,000 per month with additional benefits."

# Perform zero-shot classification
result = classifier(text, candidate_labels=candidate_labels)

# Output the result
print(result)

In [ ]:
# Define the function to detect salary data using zero-shot classification
def detect_salary_data(text):
    result = classifier(text, candidate_labels=candidate_labels)
    # Get the index of the highest score
    predicted_class_idx = result['scores'].index(max(result['scores']))
    # Return the corresponding label with the highest score
    return result['labels'][predicted_class_idx]


In [ ]:
credited_filter['model_salary_prediction'] = credited_filter.body.apply(detect_salary_data)

In [ ]:
credited_filter

In [ ]:
detect_salary_data(credited_filter.iloc[2].body)

In [ ]:
credited_filter['salary_prediction'] = credited_filter[credited_filter.flag_greater_than_10000].body.apply(detect_salary_data)

In [ ]:
%%bigquery df

SELECT
    DISTINCT mp_event_name
FROM
    `abcd-dataplatform-prod.abcd_mixpanel_raw.abcd_mp_master_event`
WHERE
    cardtitle = 'life_insurance'

In [ ]:
df

# Digitap Deep Dive

In [ ]:
%%bigquery raw_data

WITH DigitapData AS (
    SELECT 
        user_id, 
        year_month,
        invests.type,
        invests.amount
    FROM 
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS base,
        UNNEST(balances.investment) AS invests
    WHERE
        invests.type = 'EMPLOYEE PROVIDENT FUND'
)

SELECT
    *
FROM
    DigitapData

In [ ]:
raw_data.info()

In [ ]:
raw_data.user_id.nunique()

In [ ]:
grouped_data = raw_data.groupby(['user_id', 'year_month'])
grouped_data.ngroups

In [ ]:
grouped_data.amount.count()[grouped_data.amount.count() > 1]

In [ ]:
raw_data[raw_data.user_id == '6000151859'].sort_values('year_month')

In [ ]:
raw_data.type.value_counts(dropna=False)

In [ ]:
test_result = grouped_data.agg(lambda x: x.mode().iat[0] if len(x.mode()) > 0 else None)

In [ ]:
test_result

In [ ]:
test_result.reset_index()

In [ ]:
%%bigquery customer_details

SELECT
    DISTINCT id,
    mobilenumber
FROM
    `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer`
WHERE
    mobilenumber IS NOT NULL

In [ ]:
customer_details.info()

In [ ]:
results = test_result.reset_index()

In [ ]:
hits = results[results.user_id.isin(customer_details.mobilenumber.tolist())]
hits.info()

In [ ]:
hit_ratio = hits[hits.amount.str.replace(',', '').astype(float) > 1800].user_id.nunique() / customer_details.id.nunique()

In [ ]:
hit_count = hits[hits.amount.str.replace(',', '').astype(float) > 1800].user_id.nunique()

In [ ]:
f"{hit_ratio * 100:.2f}%"

In [ ]:
hits['float_amount'] = hits.amount.str.replace(',', '').astype(float)

In [ ]:
valid_hits_count = hits[
    (hits.float_amount > 1800)
    & (hits.float_amount <= 100000)
].user_id.nunique()

In [ ]:
valid_hits_ratio = valid_hits_count / customer_details.id.nunique()
valid_hits_ratio

# Lookalike asks

In [ ]:
%%bigquery appsflyer_columns

SELECT
    column_name,
    data_type
FROM
    `abcd-dataplatform-prod.abcd_curated_zone.INFORMATION_SCHEMA.COLUMNS`
WHERE
    table_name = 'fact_appsflyer_metrics_updated'

In [ ]:
appsflyer_columns[appsflyer_columns.column_name.isin(['campaign', 'campaign_id', 'adset', 'adset_id'])]

In [ ]:
campaign_ids = [
    '120213540305180000',
    '120213540305180000',
    '120211323352620000',
    '120211322825900000'
]
campaign_ids_query = ', '.join([f"'{id}'" for id in campaign_ids])
campaign_ids_query

In [ ]:
campaign_names = [
    'abcd-hl-ip-fb-leadform-ms-nov24',
    'abcd-hl-ip-fb-leadform-ms-nov24',
    'abcd_marketing_segment_medium_appinstall_signup_sept24',
    'abcd_marketing_segment_high_appinstall_signup_sept24'
]
campaign_names_query = ', '.join([f"'{id}'" for id in campaign_names])
campaign_names_query

In [ ]:
adset_ids = [
    '120213540757720000',
    '120213540305250000',
    '120211323352600000',
    '120211322825980000'
]
adset_ids_query = ', '.join([f"'{id}'" for id in adset_ids])
adset_ids_query

In [ ]:
adset_names = [
    'abcd-hl-ip-fb-leadform-ms-med',
    'abcd-hl-ip-fb-leadform-ms-high',
    'abcd_marketing_segment_medium_appinstall_signup_sept24',
    'abcd_marketing_segment_high_appinstall_signup_sept24'
]
adset_names_query = ', '.join([f"'{id}'" for id in adset_names])
adset_names_query

In [ ]:
QUERY = f"""
SELECT
    campaign,
    MAX(created_at) AS max_created_date,
    MAX(modified_at) AS max_modified_date,
    COUNT(appsflyer_id) AS row_count
FROM
    `abcd-dataplatform-prod.abcd_curated_zone.fact_appsflyer_metrics_updated`
WHERE
    campaign IN ({campaign_names_query})
GROUP BY
    campaign
"""

max_dates = client.query(QUERY).to_dataframe()
max_dates

In [ ]:
print(QUERY)

In [ ]:
QUERY = f"""
SELECT
    adset,
    MAX(created_at) AS max_created_date,
    MAX(modified_at) AS max_modified_date,
    COUNT(appsflyer_id) AS row_count
FROM
    `abcd-dataplatform-prod.abcd_curated_zone.fact_appsflyer_metrics_updated`
WHERE
    adset IN ({adset_names_query})
GROUP BY
    adset
"""

max_dates = client.query(QUERY).to_dataframe()
max_dates

In [ ]:
QUERY = f"""
WITH LatestDailyBase AS (
    SELECT
        cust_id as mobile,
        SAFE_CAST(score_v3 AS INT64) AS score,
        scrub_date,
        RANK() OVER (PARTITION BY cust_id ORDER BY scrub_date DESC) AS rank
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_SCORE_DAILY_BASE`
    WHERE
        cust_id IS NOT NULL
        AND cust_id <> 'CUSTOMER_ID'
        AND score_v3 IS NOT NULL
    QUALIFY
        rank = 1
),

LatestAdhocBase AS (
    SELECT
        cust_id as mobile,
        SAFE_CAST(score_v3 AS INT64) AS score,
        scrub_date,
        RANK() OVER (PARTITION BY cust_id ORDER BY scrub_date DESC) AS rank
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_SCORE_ADHOC_BASE` AS adhoc_base
    WHERE
        cust_id IS NOT NULL
        AND cust_id <> 'CUSTOMER_ID'
        AND score_v3 IS NOT NULL
    QUALIFY
        rank = 1
)


SELECT
    daily_base.mobile,
    daily_base.score AS daily_credit_score,
    daily_base.scrub_date AS daily_scrub_date,
    adhoc_base.score AS adhoc_credit_score,
    adhoc_base.scrub_date AS adhoc_scrub_date    
FROM
    LatestDailyBase AS daily_base
JOIN
    LatestAdhocBase AS adhoc_base
ON
    daily_base.mobile = adhoc_base.mobile
    AND daily_base.scrub_date = adhoc_base.scrub_date
"""

scores = client.query(QUERY).to_dataframe()
scores.info()

In [ ]:
scores['difference'] = scores.adhoc_credit_score - scores.daily_credit_score
bad_clashes = scores[scores.difference != 0]

In [ ]:
bad_clashes.to_csv('adhoc_daily_bad_clashes.csv', index=False)

In [ ]:
bad_clashes.difference.min(), bad_clashes.difference.max(), bad_clashes.difference.mean()

In [ ]:
scores.mobile.nunique(), scores.shape[0]

In [ ]:
mobile_counts = scores.mobile.value_counts(dropna=False)

In [ ]:
odd_mobiles = mobile_counts[mobile_counts > 1].index.tolist()

In [ ]:
scores[scores.mobile.isin(odd_mobiles)].sort_values('mobile')

In [ ]:
test = scores[scores.mobile.isin(odd_mobiles)].groupby('mobile').agg({
    'credit_score': ['mean', 'max']
})

In [ ]:
columns_names = []
for i in range(len(test.columns)):
    columns_names.append(f"{test.columns[i][1]}_{test.columns[i][0]}")
test.columns = columns_names

In [ ]:
test['difference'] = test.max_credit_score - test.mean_credit_score
test[test.difference != 0]

In [ ]:
test[test.difference == 0]

In [ ]:
bad_value_customers = test[test.difference != 0].index.tolist()
bad_value_customers_df = pd.DataFrame(bad_value_customers, columns=['customer_id'])
bad_value_customers_df.to_csv('bad_value_customers.csv', index=False)

In [ ]:
%%bigquery test_df

WITH LatestAdhocBase AS (
    SELECT
        cust_id AS mobile,
        score_v3,
        scrub_date,
        RANK() OVER (PARTITION BY cust_id ORDER BY scrub_date DESC) AS rank
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_SCORE_ADHOC_BASE` AS adhoc_base
    QUALIFY
        rank = 1
)

SELECT
    *
FROM
    LatestAdhocBase

In [ ]:
test_df[test_df.mobile == 'CUSTOMER_ID']

In [ ]:
%%bigquery odd_columns_df

WITH LatestAdhocBase AS (
    SELECT
        customer_id,
        scrub_date
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_ADHOC_BASE_UPDATED`
    WHERE
        customer_id = 'CUSTOMER_ID'
)

SELECT
    *
FROM
    LatestAdhocBase
ORDER BY
    scrub_date DESC

In [ ]:
odd_columns_df

# ETB Tagging Mobile Numbers

In [ ]:
QUERY = f"""
SELECT
    DISTINCT RIGHT(MOBILE, 10) AS mobilenumber
FROM
    `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`
"""

mobilenums = client.query(QUERY).to_dataframe()

In [ ]:
chunk_size = 900000

# Loop through chunks and write to excel
for i, start_row in enumerate(range(0, len(mobilenums), chunk_size)):
    chunk = mobilenums.iloc[start_row:start_row + chunk_size]
    file_name = f"mobile_chunk_{i + 1}.xlsx"
    chunk.to_excel(file_name, index=False)
    print(f"Saved {file_name}")

In [ ]:
QUERY = f"""
SELECT 
    CAST(id AS STRING) AS customer_id,
    installation_type,
    CASE WHEN installation_type IN ('uninstalled', 'NA') THEN 1 ELSE 0 END AS installed_flag
FROM
    (
        SELECT 
            mdn,
            CASE WHEN installation_type IS NULL THEN 'NA' ELSE installation_type END AS installation_type,
            created_date
        FROM
            (
                SELECT 
                    mdn,
                    installation_type,
                    created_date,
                    ROW_NUMBER() OVER (PARTITION BY mdn ORDER BY created_date DESC) AS row_number
                FROM 
                    `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer_activation`
            )
        WHERE row_number = 1
    ) ac
LEFT JOIN 
    `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer` tc
ON ac.mdn = tc.mobilenumber
"""

installed_customers = client.query(QUERY).to_dataframe()
installed_customers.info()

In [ ]:
installed_customers.installed_flag.value_counts()

In [ ]:
# Select Target
target = 'pl'

# Variables for Internal Tables
internal_project = 'abcd-dataplatform'
internal_dataset = 'abcd_data_science_app'
internal_table = f'{target}_explore_users_internal'
temp_merger_table = f'{target}_explore_users_merger_intermediate'

# Variables for External Tables
external_project = 'abcd-dataplatform-prod'
external_dataset = 'abcd_analytics_ml_usecase_downstream_consumption'
external_table = f'{target}_explore_users'
external_table_high_cs = f'{target}_explore_users_high_credit_score'
external_table_no_cs = f'{target}_explore_users_no_credit_score'

In [ ]:
QUERY = f"""
WITH installed_customer_flags AS (
    SELECT 
        CAST(id AS STRING) AS customer_id,
        installation_type,
        CASE WHEN installation_type IN ('uninstalled', 'NA') THEN 1 ELSE 0 END AS installed_flag
    FROM
        (
            SELECT 
                mdn,
                CASE WHEN installation_type IS NULL THEN 'NA' ELSE installation_type END AS installation_type,
                created_date
            FROM
                (
                    SELECT 
                        mdn,
                        installation_type,
                        created_date,
                        ROW_NUMBER() OVER (PARTITION BY mdn ORDER BY created_date DESC) AS row_number
                    FROM 
                        `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer_activation`
                )
            WHERE row_number = 1
        ) ac
    LEFT JOIN 
        `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer` tc
    ON ac.mdn = tc.mobilenumber
)

SELECT
    external_table.customer_id,
    CAST(CAST(external_table.mobilenumber AS INT64) AS STRING) AS mobilenumber,
    internal_table.decile,
    internal_table.probability_score AS probability_score,
    internal_table.group,
    internal_table.group_decile,
    internal_table.sent,
    external_table.upload_date AS external_upload_date,
    internal_table.upload_date AS internal_upload_date,
    CASE WHEN icf.installed_flag IS NULL THEN 0 ELSE icf.installed_flag END AS installation_flag
FROM
    `{external_project}.{external_dataset}.{external_table_high_cs}` AS external_table
LEFT JOIN
    `{internal_project}.{internal_dataset}.{internal_table}` AS internal_table
ON
    internal_table.customer_id = external_table.customer_id
    AND internal_table.upload_date = CAST(external_table.upload_date AS DATETIME)
LEFT JOIN
    installed_customer_flags AS icf
ON
    external_table.customer_id = icf.customer_id
WHERE
    CAST(external_table.upload_date AS DATETIME) >= '2024-12-01'
"""

external_table_df = client.query(QUERY).to_dataframe()
external_table_df.info(memory_usage='deep')

In [ ]:
print(f"{external_table_df.installation_flag.sum() * 100 / external_table_df.shape[0]:.2f} %")

In [ ]:
(70075, 142196)

In [ ]:
test_results = pd.merge(
    external_table_df,
    installed_customers,
    on='customer_id',
    how='left'
)
test_results.info()

In [ ]:
test_results.groupby(['group', 'group_decile']).agg({'installed_flag': ['sum', 'count']})

In [ ]:
QUERY = f"""
SELECT 
    CAST(RIGHT(phone, 10) AS STRING) AS mobilenumber,
    1 AS reachability_flag
FROM 
    `abcd-dataplatform-prod.abcd_clevertap_batch_raw.user_profile_raw`
WHERE 
    clevertapProfileAttr_MSG_push = 'true'
"""

clevertap_reachability = client.query(QUERY).to_dataframe()
clevertap_reachability.info()

In [ ]:
final = pd.merge(
    test_results,
    clevertap_reachability,
    on='mobilenumber',
    how='left'
)

In [ ]:
final['reachability_flag'] = final.reachability_flag.fillna(0)

In [ ]:
final.groupby(['group', 'group_decile']).agg({'reachability_flag': ['sum', 'count']})

# Alternate Credit Scoring

In [ ]:
year = 2024
month = 4

QUERY = f"""
SELECT
    CAST(id AS STRING) customer_id,
    mobilenumber,
    createddate
FROM
    `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer`
WHERE
    EXTRACT(MONTH FROM createddate) = {month}
    AND EXTRACT(YEAR FROM createddate) = {year}
"""

app_month_registration = client.query(QUERY).to_dataframe()
app_month_registration.info()

In [ ]:
column_list = [
    "CUSTOMER_ID AS mobilenumber", 
    "ACCT_KEY",
    "ACCT_TYPE_CD", 
    "SAFE_CAST(DAYS_PAST_DUE_01 AS FLOAT64) AS DAYS_PAST_DUE_01",
    "SAFE_CAST(DAYS_PAST_DUE_02 AS FLOAT64) AS DAYS_PAST_DUE_02",
    "SAFE_CAST(DAYS_PAST_DUE_03 AS FLOAT64) AS DAYS_PAST_DUE_03",
    "SAFE_CAST(DAYS_PAST_DUE_04 AS FLOAT64) AS DAYS_PAST_DUE_04",
    "SAFE_CAST(DAYS_PAST_DUE_05 AS FLOAT64) AS DAYS_PAST_DUE_05",
    "SAFE_CAST(DAYS_PAST_DUE_06 AS FLOAT64) AS DAYS_PAST_DUE_06",
    "SAFE_CAST(DAYS_PAST_DUE_07 AS FLOAT64) AS DAYS_PAST_DUE_07",
    "SAFE_CAST(DAYS_PAST_DUE_08 AS FLOAT64) AS DAYS_PAST_DUE_08",
    "SAFE_CAST(DAYS_PAST_DUE_09 AS FLOAT64) AS DAYS_PAST_DUE_09",
    "SAFE_CAST(DAYS_PAST_DUE_10 AS FLOAT64) AS DAYS_PAST_DUE_10",
    "SAFE_CAST(DAYS_PAST_DUE_11 AS FLOAT64) AS DAYS_PAST_DUE_11",
    "SAFE_CAST(DAYS_PAST_DUE_12 AS FLOAT64) AS DAYS_PAST_DUE_12",
    "SAFE.PARSE_DATE('%d/%m/%Y', OPEN_DT) AS OPEN_DT", 
    "SAFE_CAST(BALANCE_AM AS FLOAT64) AS BALANCE_AM", 
    "SAFE.PARSE_DATE('%d/%m/%Y', BALANCE_DT) AS BALANCE_DT", 
    "SAFE.PARSE_DATE('%d/%m/%Y', CLOSED_DT) AS CLOSED_DT", 
    "SAFE_CAST(CREDIT_LIMIT_AM AS FLOAT64) AS CREDIT_LIMIT_AM", 
    "SAFE_CAST(ORIG_LOAN_AM AS FLOAT64) AS ORIG_LOAN_AM",
    "SCRUB_DATE"
]
columns_query = ',\n'.join(column_list)

In [ ]:
updated_column_list = [
    "mobilenumber", 
    "ACCT_KEY", 
    "ACCT_TYPE_CD", 
    "DAYS_PAST_DUE_01",
    "DAYS_PAST_DUE_02",
    "DAYS_PAST_DUE_03",
    "DAYS_PAST_DUE_04",
    "DAYS_PAST_DUE_05",
    "DAYS_PAST_DUE_06",
    "DAYS_PAST_DUE_07",
    "DAYS_PAST_DUE_08",
    "DAYS_PAST_DUE_09",
    "DAYS_PAST_DUE_10",
    "DAYS_PAST_DUE_11",
    "DAYS_PAST_DUE_12",
    "OPEN_DT",
    "BALANCE_AM", 
    "BALANCE_DT", 
    "CLOSED_DT", 
    "CREDIT_LIMIT_AM", 
    "ORIG_LOAN_AM",
    "SCRUB_DATE"
]
updated_columns_query = ', \n'.join(updated_column_list)

In [ ]:
year = 2024
month = 4

current_date = f'{year}-{month:02d}-01'  # Format to 'YYYY-MM-01'

QUERY = f"""
WITH customer_base AS (
    SELECT
        CAST(id AS STRING) customer_id,
        mobilenumber,
        createddate
    FROM
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer`
    WHERE
        EXTRACT(MONTH FROM createddate) = {month}
        AND EXTRACT(YEAR FROM createddate) = {year}
),

latest_data AS (
    SELECT
        RANK() OVER (PARTITION BY mobilenumber, ACCT_KEY ORDER BY source ASC, SCRUB_DATE DESC) AS rank,
        {updated_columns_query}
    FROM (
        SELECT
            {columns_query},
            1 AS source
        FROM
            `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
        WHERE
            CUSTOMER_ID IN (SELECT mobilenumber FROM customer_base)
        
        UNION ALL
        
        SELECT
            {columns_query},
            0 AS source
        FROM
            `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_ADHOC_BASE_UPDATED`
        WHERE
            CUSTOMER_ID IN (SELECT mobilenumber FROM customer_base)
    )
    QUALIFY rank = 1
),

quality_data AS (
    SELECT
        mobilenumber,
        ACCT_KEY,
        ACCT_TYPE_CD,
        DAYS_PAST_DUE_01,
        DAYS_PAST_DUE_02,
        DAYS_PAST_DUE_03,
        DAYS_PAST_DUE_04,
        DAYS_PAST_DUE_05,
        DAYS_PAST_DUE_06,
        DAYS_PAST_DUE_07,
        DAYS_PAST_DUE_08,
        DAYS_PAST_DUE_09,
        DAYS_PAST_DUE_10,
        DAYS_PAST_DUE_11,
        DAYS_PAST_DUE_12,
        CASE 
            WHEN OPEN_DT < DATE('1900-01-01') THEN NULL
            ELSE OPEN_DT
        END AS OPEN_DT,
        BALANCE_DT,
        CASE 
            WHEN CLOSED_DT < DATE('1899-12-30') THEN NULL
            ELSE CLOSED_DT
        END AS CLOSED_DT,
        CREDIT_LIMIT_AM, 
        ORIG_LOAN_AM AS ORIG_ORIG_LOAN_AM,
        CASE
            WHEN ORIG_LOAN_AM < 0 THEN NULL
            ELSE ORIG_LOAN_AM
        END AS ORIG_LOAN_AM,
        CASE
            WHEN BALANCE_AM < 0 THEN NULL
            ELSE BALANCE_AM
        END AS BALANCE_AM,
        SCRUB_DATE
    FROM
        latest_data
)

SELECT
    cb.*,
    acct_key,
    acct_type_cd,
    days_past_due_01,
    days_past_due_02,
    days_past_due_03,
    days_past_due_04,
    days_past_due_05,
    days_past_due_06,
    days_past_due_07,
    days_past_due_08,
    days_past_due_09,
    days_past_due_10,
    days_past_due_11,
    days_past_due_12,
    open_dt,
    closed_dt,
    balance_dt,
    credit_limit_am,
    orig_loan_am,
    balance_am,
    CASE 
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 1 AND days_past_due_02 - days_past_due_01 > 30 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 2 AND days_past_due_02 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 3 AND days_past_due_03 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 4 AND days_past_due_04 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 5 AND days_past_due_05 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 6 AND days_past_due_06 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 7 AND days_past_due_07 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 8 AND days_past_due_08 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 9 AND days_past_due_09 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 10 AND days_past_due_10 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 11 AND days_past_due_11 > 0 THEN 1
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 12 AND days_past_due_12 > 0 THEN 1
        ELSE 0
    END AS dpd_current_date,
    CASE 
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 1 AND days_past_due_01 > 0 THEN 'days_past_due_01'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 2 AND days_past_due_02 > 0 THEN 'days_past_due_02'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 3 AND days_past_due_03 > 0 THEN 'days_past_due_03'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 4 AND days_past_due_04 > 0 THEN 'days_past_due_04'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 5 AND days_past_due_05 > 0 THEN 'days_past_due_05'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 6 AND days_past_due_06 > 0 THEN 'days_past_due_06'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 7 AND days_past_due_07 > 0 THEN 'days_past_due_07'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 8 AND days_past_due_08 > 0 THEN 'days_past_due_08'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 9 AND days_past_due_09 > 0 THEN 'days_past_due_09'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 10 AND days_past_due_10 > 0 THEN 'days_past_due_10'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 11 AND days_past_due_11 > 0 THEN 'days_past_due_11'
        WHEN DATE_DIFF(balance_dt, '{current_date}', MONTH) = 12 AND days_past_due_12 > 0 THEN 'days_past_due_12'
        ELSE 'no_capture'
    END AS dpd_current_date_capture
FROM
    customer_base AS cb
LEFT JOIN
    quality_data AS exp
ON
    cb.mobilenumber = exp.mobilenumber
"""

In [ ]:
alt_cs_qj = client.query(QUERY)
alt_cs = alt_cs_qj.to_dataframe()
alt_cs.info()

In [ ]:
alt_cs[alt_cs.dpd_current_date == 1]

In [ ]:
alt_cs.balance_dt.value_counts(dropna=False).sort_index()

In [ ]:
date_cols = ['open_dt', 'closed_dt', 'balance_dt']
for col in date_cols:
    alt_cs[col] = pd.to_datetime(alt_cs[col], errors='coerce')

alt_cs_filtered = alt_cs[alt_cs[date_cols].notna().all(axis=1)]

In [ ]:
alt_cs_filtered.customer_id.nunique()

In [ ]:
alt_cs_filtered.info()

In [ ]:
alt_cs_filtered['exclusion_filter'] = np.where(
    (alt_cs_filtered.balance_dt >= '2024-04-01')
    & (alt_cs_filtered.days_past_due_01 > 0), 
    0, 1
)

alt_cs_filtered

In [ ]:
alt_cs_filtered['dpd_current_date'] = np.where(
    alt_cs_filtered['']
)

##### Store trial 50 customers

In [ ]:
test_customers = app_month_registration.head(50).customer_id.tolist()
trial_set = alt_cs[alt_cs.customer_id.isin(test_customers)]

In [ ]:
def encode_to_base64(text):
    if isinstance(text, str):  # Ensure the input is a string
        encoded_bytes = base64.b64encode(text.encode("utf-8"))  # Convert to bytes, then encode
        encoded_text = encoded_bytes.decode("utf-8")  # Convert bytes back to string
        return encoded_text
    else:
        return None  # Return None for invalid or missing input

In [ ]:
def decode_from_base64(encoded_text):
    if isinstance(encoded_text, str):  # Ensure the input is a string
        try:
            decoded_bytes = base64.b64decode(encoded_text)  # Decode to bytes
            decoded_text = decoded_bytes.decode("utf-8")  # Convert bytes back to string
            return decoded_text
        except Exception as e:
            print(f"Decoding failed: {e}")
            return None  # Return None or a placeholder for decoding failure
    else:
        return None  # Return None if the input is not a string (e.g., float or NaN)

In [ ]:
trial_set['mobilenumber'] = trial_set['mobilenumber'].apply(encode_to_base64)
trial_set.to_csv('Alternate_credit_scoring_test_customers.csv', index=False)
trial_set

# Experian DAILY and ADHOC base customer hits (CUSTOMER SEGMENTATION)

In [ ]:
QUERY = f"""
SELECT
    customer_id AS mobilenumber,
    gender
FROM
    `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_NAME_DOB_DAILY_BASE`
"""

test = client.query(QUERY).to_dataframe()
test.shape

In [ ]:
test.mobilenumber.nunique()

In [ ]:
QUERY = f"""
SELECT 
  DISTINCT CUSTOMER_ID
FROM 
  `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_ADHOC_BASE_UPDATED`
WHERE
  CUSTOMER_ID IN (
    SELECT MOBILE FROM `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`
  )
"""

adhoc_customer_hits = client.query(QUERY).to_dataframe()
adhoc_customer_hits.info()

In [ ]:
QUERY = f"""
SELECT 
  DISTINCT CUSTOMER_ID
FROM 
  `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_ADHOC_BASE_UPDATED`
WHERE
  REMARKS = 'APP_SIGN_UP'
  AND CUSTOMER_ID IN (
    SELECT MOBILE FROM `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`
  )
"""

adhoc_customer_app_signup_hits = client.query(QUERY).to_dataframe()
adhoc_customer_app_signup_hits.info()

In [ ]:
QUERY = f"""
SELECT 
  DISTINCT CUSTOMER_ID
FROM 
  `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
WHERE
  CUSTOMER_ID IN (
    SELECT MOBILE FROM `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`
  )
"""

daily_customer_hits = client.query(QUERY).to_dataframe()
daily_customer_hits.info()

In [ ]:
common_customer_ids = adhoc_customer_hits[
    adhoc_customer_hits.CUSTOMER_ID.isin(
        daily_customer_hits.CUSTOMER_ID.tolist()
    )
]
common_customer_ids.shape

In [ ]:
unique_adhoc_customer_ids = adhoc_customer_hits[
    ~adhoc_customer_hits.CUSTOMER_ID.isin(
        daily_customer_hits.CUSTOMER_ID.tolist()
    )
]
unique_adhoc_customer_ids.shape

In [ ]:
unique_daily_customer_ids = daily_customer_hits[
    ~daily_customer_hits.CUSTOMER_ID.isin(
        adhoc_customer_hits.CUSTOMER_ID.tolist()
    )
]
unique_daily_customer_ids.shape

In [ ]:
adhoc_customer_hits.to_csv('experian_analysis/adhoc_customer_hits.csv', index=False)
adhoc_customer_app_signup_hits.to_csv('experian_analysis/adhoc_customer_app_signup_hits.csv', index=False)
daily_customer_hits.to_csv('experian_analysis/daily_customer_hits.csv', index=False)

In [ ]:
adhoc_customer_app_signup_hits.CUSTOMER_ID.isin(unique_adhoc_customer_ids.CUSTOMER_ID.tolist()).sum()

# SMS Deep Dive (Again :rolling_eyes:)

In [ ]:
QUERY = f"""
SELECT
  year_month,
  COUNT(DISTINCT year_month) as outer_yearmonth_count,
  COUNT(DISTINCT balances.year_month) AS balance_yearmonth_count
FROM
  `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`
GROUP BY
    year_month
ORDER BY
    year_month DESC
"""

test = client.query(QUERY).to_dataframe()
test

In [ ]:
QUERY = f"""
SELECT
  base.user_id,
  base.year_month AS base_year_month,
  base.balances.year_month AS balances_year_month,
  base.balances AS balances_struct,
  ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY base.year_month DESC) AS row_num
FROM
  `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS base
QUALIFY row_num = 1
"""

test = client.query(QUERY).to_dataframe()
test

In [ ]:
test.info(memory_usage='deep')

In [ ]:
test.to_pickle('sms_raw_data.pkl')

In [ ]:
test[
    (test.balances_year_month.notna())
    # & (test.balances_year_month != test.base_year_month)
]

In [ ]:
QUERY = f"""
SELECT
    customer_id,
    mobilenumber
FROM
    `abcd-dataplatform.abcd_data_model.user_activity_raw`
WHERE
    mobilenumber = '6398190801'
"""

trial = client.query(QUERY).to_dataframe()
trial

In [ ]:
QUERY = f"""
WITH bank_accounts_flattened AS (
    SELECT
        base.user_id,
        ARRAY_AGG(DISTINCT bank_accounts.bank) AS unique_banks
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS base,
        UNNEST(base.balances.bank_accounts) AS bank_accounts
    GROUP BY
        base.user_id
),

loan_dues_flattened AS (
    SELECT
        base.user_id,
        ARRAY_AGG(loan_dues.bank) AS unique_banks
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS base,
        UNNEST(base.balances.loan_dues) AS loan_dues
    GROUP BY
        base.user_id
),

credit_card_flattened AS (
    SELECT
        base.user_id,
        ARRAY_AGG(credit_card.bank) AS unique_banks
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS base,
        UNNEST(base.balances.credit_card_accounts) AS credit_card
    GROUP BY
        base.user_id
),

joined_data AS (
    SELECT
        DISTINCT base.user_id,
        bank_accounts.unique_banks AS banks_from_accounts,
        loan_dues.unique_banks AS banks_from_loan_details,
        credit_card.unique_banks AS banks_from_credit_card
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS base
    LEFT JOIN
        bank_accounts_flattened AS bank_accounts
    ON
        base.user_id = bank_accounts.user_id
    LEFT JOIN
        loan_dues_flattened AS loan_dues
    ON
        base.user_id = loan_dues.user_id
    LEFT JOIN
        credit_card_flattened AS credit_card
    ON
        base.user_id = credit_card.user_id
)

SELECT
    * 
FROM
    joined_data
"""

In [ ]:
bank_details_df = client.query(QUERY).to_dataframe()
bank_details_df.info()

In [ ]:
bank_details_df.head()

## Credit Limit

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'Apr-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        credit_card.due_amount,
        credit_card.due_date,
        credit_card.repayment,
        credit_card.repay_date,
        credit_card.repayment_pm,
        credit_card.repay_date_pm,
        credit_card.total_credit_limit,
        credit_card.status
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
        UNNEST(balances.credit_card_accounts) AS credit_card
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        SAFE_CAST(sms.due_amount AS FLOAT64) AS due_amount,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.due_date) AS due_date,
        SAFE_CAST(sms.repayment AS FLOAT64) AS repayment,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.repay_date) AS repay_date,
        SAFE_CAST(sms.repayment_pm AS FLOAT64) AS repayment_pm,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.repay_date_pm) AS repay_date_pm,
        SAFE_CAST(sms.total_credit_limit AS FLOAT64) AS total_credit_limit,
        sms.status
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sms_period_mark DESC) AS rank
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
    QUALIFY rank = 1
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    total_credit_limit,
    due_amount,
    due_date,
    repayment,
    repay_date,
    repayment_pm,
    repay_date_pm,
    status
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

In [ ]:
credit_scoring_analysis.status.value_counts(dropna=False)

In [ ]:
credit_scoring_analysis[credit_scoring_analysis.status == 'fully paid']

In [ ]:
credit_scoring_analysis.groupby(['year_month']).agg({
    'customer_id': ['count', 'nunique']
})

In [ ]:
credit_scoring_analysis[credit_scoring_analysis.total_credit_limit > 0].total_credit_limit.value_counts(dropna=False)

In [ ]:
credit_scoring_analysis[credit_scoring_analysis.total_credit_limit > 0].customer_id.nunique()

In [ ]:
credit_scoring_analysis.customer_id.value_counts(dropna=False)

## Salary

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'Sep-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        sms_base.total_salary_credit
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        SAFE_CAST(sms.total_salary_credit AS FLOAT64) AS total_salary_credit
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    total_salary_credit
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

In [ ]:
credit_scoring_analysis[credit_scoring_analysis.total_salary_credit > 0].customer_id.nunique()

In [ ]:
agg_data = credit_scoring_analysis.groupby(['customer_id', 'year_month']).agg({
    'total_salary_credit': ['max', 'mean', 'count']
})
agg_data.columns = ['salary_max', 'salary_mean', 'salary_count']

agg_data[
    (agg_data['salary_count'] > 1)
    & (agg_data.salary_max > 0)
].reset_index().customer_id.nunique()

## ITR

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'Apr-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        itr.itr_type,
        itr.amount
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
        UNNEST(balances.itr) AS itr
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        sms.itr_type,
        SAFE_CAST(sms.amount AS FLOAT64) AS itr_filed_amount
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sms_period_mark DESC) AS rank
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
    QUALIFY rank = 1
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    itr_type,
    itr_filed_amount
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

In [ ]:
credit_scoring_analysis.itr_type.value_counts(normalize=True)

## Top Spend Areas

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'May-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        top_spend.category,
        top_spend.amount
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
        UNNEST(top_spend_areas) AS top_spend
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        sms.category,
        SAFE_CAST(sms.amount AS FLOAT64) AS top_spend_amount
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sms_period_mark DESC) AS rank
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
    QUALIFY rank = 1
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    category,
    top_spend_amount
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

## Monthly Transactions

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'Aug-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        top_spend.category,
        top_spend.amount
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
        UNNEST(monthly_transactions) AS top_spend
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        sms.category,
        SAFE_CAST(sms.amount AS FLOAT64) AS spend_amount
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sms_period_mark DESC) AS rank
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
    QUALIFY rank = 1
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    category,
    spend_amount
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

## EMI dues

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'Apr-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        loan_dues.emi_due,
        loan_dues.emi_date,
        loan_dues.emi_paid,
        loan_dues.emi_paid_date,
        loan_dues.loan_type
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
        UNNEST(balances.loan_dues) AS loan_dues
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        SAFE_CAST(sms.emi_due AS FLOAT64) as emi_due,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.emi_date) AS emi_date,
        SAFE_CAST(sms.emi_paid AS FLOAT64) as emi_paid,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.emi_paid_date) AS emi_paid_date,
        sms.loan_type
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sms_period_mark DESC) AS rank
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
    QUALIFY rank = 1
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    emi_due,
    emi_date,
    emi_paid,
    emi_paid_date,
    loan_type
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

## Utility dues

In [ ]:
QUERY = f"""
WITH credit_score_base AS (
    SELECT
        snapshot_month,
        customer_id,
        mobilenumber
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
    WHERE
        snapshot_month = 'Sep-2024'
),

filtered_sms_data AS (
    SELECT
        sms_base.user_id,
        sms_base.year_month,
        utilities.paid_amount,
        utilities.paid_date,
        utilities.due_amount,
        utilities.due_date,
        utilities.utility_type
    FROM
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
        UNNEST(balances.utility) AS utilities
    WHERE
        user_id IN (SELECT mobilenumber FROM credit_score_base)
),

joined_data AS (
    SELECT
        base.customer_id,
        base.mobilenumber,
        base.snapshot_month,
        SAFE.PARSE_DATE('%b-%Y', base.snapshot_month) AS snapshot_period_mark,
        sms.year_month,
        SAFE.PARSE_DATE('%Y-%m', sms.year_month) AS sms_period_mark,
        SAFE_CAST(sms.due_amount AS FLOAT64) as due_amount,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.due_date) AS due_date,
        SAFE_CAST(sms.paid_amount AS FLOAT64) as paid_amount,
        SAFE.PARSE_DATE('%Y-%m-%d', sms.paid_date) AS paid_date,
        sms.utility_type
    FROM
        credit_score_base AS base
    LEFT JOIN
        filtered_sms_data AS sms
    ON
        base.mobilenumber = sms.user_id
),

filtered_data AS (
    SELECT
        joined_base.*,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sms_period_mark DESC) AS rank
    FROM
        joined_data AS joined_base
    WHERE
        DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) <= 6
        AND DATE_DIFF(snapshot_period_mark, sms_period_mark, MONTH) >= 0
    QUALIFY rank = 1
)

SELECT
    customer_id,
    mobilenumber,
    snapshot_month,
    snapshot_period_mark,
    year_month,
    sms_period_mark,
    paid_amount,
    paid_date,
    due_amount,
    due_date,
    utility_type
FROM
    filtered_data
ORDER BY
    customer_id ASC,
    snapshot_month ASC,
    year_month DESC
"""

credit_scoring_analysis = client.query(QUERY).to_dataframe()
credit_scoring_analysis.info(memory_usage='deep')

In [ ]:
credit_scoring_analysis.due_amount.value_counts(dropna=False)

In [ ]:
QUERY = f"""
SELECT
  DISTINCT credit_card.status
FROM
  `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`,
  UNNEST(balances.credit_card_accounts) AS credit_card
WHERE
    CAST(credit_card.status AS STRING) IS NOT NULL
"""

test = client.query(QUERY).to_dataframe()
test

# Nupur & Shriya Queries

In [ ]:
base ="""
 
with base as
(select distinct mobilenumber,occupation, "Bureau" as tag
from `abcd-dataplatform.abcd_data_model.user_activity_raw_backup_06_jan_2025` a)
 
,sms as
(Select distinct user_id as mobile,'SALARIED' as occupation, "Sms" as tag
from abcd-dataplatform.abcd_aggregated_zone.sms_sal_epfo )
 
,enuiry_base as
(Select distinct CUSTOMER_ID as mobile_number,INQ_PURP_CD,
PARSE_DATE('%d/%m/%Y', INQ_DATE) as inquiry_date 
from abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ENQ_DAILY_BASE
where INQ_PURP_CD='14')
 
,salaried_final_base as(
select c.customer_id,c.occupation,c.mobilenumber,
d.final_uninstall_flag,d.credit_score,d.pl_purchase_flag,
d.hfl_purchase_flag,e.inquiry_date,
case when c.mobilenumber = e.mobile_number then 1 else 0 end as enq_flag
from
(select b.customer_id,a.mobilenumber,a.occupation
from
(select mobilenumber,occupation
from 
(select mobilenumber,occupation, tag,
row_number() over (partition by mobilenumber order by tag desc) as rw
from 
(Select distinct mobilenumber,occupation, tag
from base
union all 
Select distinct mobile,occupation, tag 
from sms))
where rw = 1) a
inner join `abcd-dataplatform.abcd_data_model.user_activity_raw_backup_06_jan_2025` b
on a.mobilenumber = b.mobilenumber) c
left join
`abcd-dataplatform.abcd_data_model.user_activity_raw_backup_06_jan_2025` d
on c.mobilenumber = d.mobilenumber
left join
enuiry_base e
on c.mobilenumber = e.mobile_number
)
 
 
select 
distinct customer_id,
case when final_uninstall_flag = 0 then 1 else 0 end as active_flag
,"HL" as product,"CO_2" as cohort
from salaried_final_base a
where a.occupation <> "SALARIED" 
and credit_score in ('700-750','750-800','>800')
and hfl_purchase_flag = 0 
and enq_flag = 1 
and left(cast(date(inquiry_date) as string),7) in 
("2024-12","2024-11","2024-10","2024-09","2024-08","2024-07")
"""

df = client.query(base).to_dataframe()
df.info()

In [ ]:
base = f"""
select
distinct a.customer_id,
case when final_uninstall_flag = 1 then 0 else 1 end as active_flag
,"MI" as product,"CO_1" as cohort
from `abcd-dataplatform.abcd_data_model.user_activity_raw_backup_06_jan_2025` a
inner join
(select distinct CUSTOMER_ID as mobile_number,ACCT_TYPE_CD as acc_type,
format_date('%Y-%m-%d', PARSE_DATE('%d/%m/%Y', OPEN_DT)) as starting_date,CLOSED_DT as closing_date,
format_date('%m', parse_date('%d/%m/%Y', OPEN_DT)) as starting_month
from abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED
where ACCT_TYPE_CD in ('221','246','173')
and length(CAST(CUSTOMER_ID as string)) = 10
) b
on a.mobilenumber = b.mobile_number;
"""

df = client.query(base).to_dataframe()
df.info()

In [ ]:
pandas_gbq.to_gbq(
    dataframe=df,
    destination_table='abcd_data_model.d2c_target_campaign_data',
    project_id='abcd-dataplatform',
    if_exists='append'
)

In [ ]:
df.active_flag.sum()

In [ ]:
QUERY = f"""
SELECT
    customer_id,
    product,
    cohort
FROM
    `abcd-dataplatform.abcd_data_model.d2c_target_campaign_data`
"""

test_df = client.query(QUERY).to_dataframe()
test_df.info()

In [ ]:
drop_dupe_df = test_df.drop_duplicates()
drop_dupe_df.info()

In [ ]:
grouped = drop_dupe_df.groupby(['product', 'cohort']).agg({
    'customer_id': ['count', 'nunique']
})
grouped.columns = ['row_count', 'cust_count']
grouped

In [ ]:
grouped[
    (grouped.cust_count != grouped.row_count)
]

# Anagog (Yaaaaay....?)

In [ ]:
QUERY = f"""
SELECT
    customerid AS customer_id,
    PremiumPotential,
    CarBrandName,
    CarModelName,
    PhoneBrandName,
    PhoneModelName
FROM
    `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
WHERE
    flag = 1
"""

test = client.query(QUERY).to_dataframe()
test

In [ ]:
test.PhoneBrandName.unique()

In [ ]:
test.CarBrandName.unique()

In [ ]:
test[
    (test.PremiumPotential >= 0.05)
    & (test.CarBrandName.notna())
    & (test.CarModelName.notna())
    & (test.CarBrandName != 'Unknown')
    & (test.PhoneBrandName.notna())
]

In [ ]:
test[
    (test.CarBrandName == 'BMW')
]

In [ ]:
test['FlagPremiumPotential'] = np.where(
    (test.PremiumPotential >= 0.05),
    1, 0
)
test['FlagPremiumPotential'].value_counts(normalize=True)

In [ ]:
QUERY = f"""
SELECT
    id,
    firstname,
    lastname,
    mobilenumber
FROM
    `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer`
WHERE
    id = 1941961
"""

trial_cust = client.query(QUERY).to_dataframe()
trial_cust

In [ ]:
test[test.customer_id == '4244434']

In [ ]:
QUERY = f"""
SELECT
    customerid AS customer_id,
    ingestion_time,
    * EXCEPT(customerid, ingestion_time)
FROM
    `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
WHERE
    customerid = '4244434'
"""
snippet = client.query(QUERY).to_dataframe()
snippet

In [ ]:
snippet.raw_ingestion_time

In [ ]:
different_columns = snippet.loc[:, snippet.fillna('___NaN___').nunique() > 1].columns.tolist()

In [ ]:
different_columns

In [ ]:
cols_list = ['customer_id']
cols_list.extend(different_columns)
cols_list

In [ ]:
snippet[cols_list].sort_values('raw_ingestion_time').reset_index(drop=True).to_csv('Anagog_snippet.csv', index=False)

In [ ]:
QUERY = f"""
SELECT
    customerid AS customer_id,
    ingestion_time,
    CurrentBestMoment,
    PageLastVisit,
    PageLastVisitDate,
    raw_ingestion_time,
    ActivityDate,
    flag
FROM
    `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
WHERE
    customerid = '4244434'
ORDER BY
    raw_ingestion_time ASC
"""

queried_snippet = client.query(QUERY).to_dataframe()
queried_snippet

In [ ]:
flag_cols = [
    # 'HouseholdSize',
    'IsParent',
    # 'IsParentAppCount',
    'financenewsapps',
    # 'financenewsappsappcount',
    # 'financenewsservices',
    'IsUsingDigitalPayment',
    # 'IsUsingDigitalPaymentAppCount',
    # 'BankingServices',
    'bankingapps',
    # 'bankingappsappcount',
    # 'BankingServices1',
    'IsStockInvest',
    # 'IsStockInvestAppCount',
    'investmentsapps',
    # 'investmentsappsappcount',
    # 'investmentsservices',
    'insuranceapps',
    # 'insuranceappsappcount',
    # 'insuranceservices',
    # 'NumGamesAppsInstalled',
    'CurrentlyAbroad',
    'IsHeavyDriver',
    'IsLightDriver',
    'IsTraveler',
    'IsHeavyUser',
    'IsLightUser',
    'IsTechie',
    'CarBrandName',
    'PhoneAgeDays',
    'PhoneBrandName',
    'PhoneBrandName_sdk',
    'PhoneDeviceRaw',
    'PhoneModelName',
    'PhoneModelName_sdk',
    'PhoneModelRaw',
    'PhoneModelRaw_sdk'
]

col_query = ',\n\t'.join(flag_cols)
print(col_query)

In [ ]:
QUERY = f"""
WITH credit_score_data AS (
    SELECT
        customer_id,
        snapshot_month
    FROM
        `abcd-dataplatform.abcd_data_science_app.alternate_credit_scoring_base`
),

anagog_data AS (
    SELECT
        customerid AS customer_id,
        PremiumPotential,
        ChurnProbabilityModel,
        ingestion_time
    FROM
        `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
    WHERE
        flag = 1
),

joined_data AS (
    SELECT
        cs.customer_id,
        LEFT(SAFE_CAST(SAFE.PARSE_DATE('%b-%Y', cs.snapshot_month) AS STRING), 7) AS snapshot_month,
        ad.PremiumPotential,
        ad.ChurnProbabilityModel
    FROM
        credit_score_data AS cs
    JOIN
        anagog_data AS ad
    ON
        cs.customer_id = ad.customer_id
)

SELECT
    *
FROM
    joined_data
"""

In [ ]:
raw_data = client.query(QUERY).to_dataframe()
raw_data

In [ ]:
raw_data['FlagChurnProbabilityModel'] = np.where(
    (raw_data.ChurnProbabilityModel.isna()),
    0, 1
)
raw_data['FlagChurnProbabilityModel'].value_counts()

In [ ]:
raw_data['FlagPremiumPotential'] = np.where(
    (raw_data.PremiumPotential.isna()),
    pd.NA, np.where(
        (raw_data.PremiumPotential >= 0.05),
        1, 0
    )
)
(raw_data.groupby(['snapshot_month'])['FlagPremiumPotential'].value_counts(dropna=False, normalize=False)).sort_index()

In [ ]:
mean_potential = raw_data.PremiumPotential.mean()
raw_data['FlagPremiumPotential'] = np.where(
    (raw_data.PremiumPotential.isna()),
    pd.NA, np.where(
        (raw_data.PremiumPotential >= mean_potential),
        1, 0
    )
)
(raw_data.groupby(['snapshot_month'])['FlagPremiumPotential'].value_counts(dropna=False, normalize=True) * 100).sort_index()

In [ ]:
mean_potential

In [ ]:
flagged_data = raw_data[raw_data.flag == 1]
flagged_data.head()

In [ ]:
flagged_data.IsTechie.value_counts(dropna=False, normalize=True)

In [ ]:
flagged_data['FlagPremiumPotential'] = np.where(
    (flagged_data.PremiumPotential.isna()),
    pd.NA, np.where(
        (flagged_data.PremiumPotential >= 0.05),
        1, 0
    )
)
flagged_data['FlagPremiumPotential'].value_counts(dropna=False, normalize=True)

In [ ]:
flagged_data.financenewsapps.value_counts(dropna=False, normalize=True)

In [ ]:
flagged_data.info()

In [ ]:
for col in flag_cols:
    print(flagged_data[col].value_counts(dropna=False, normalize=True))
    print()

In [ ]:
QUERY = f"""
WITH Anagog_data AS (
    SELECT
        CAST(customerid AS STRING) AS customer_id,
        ingestion_time,
        CurrentBestMoment,
        PageLastVisit,
        PageLastVisitDate,
        raw_ingestion_time,
        ActivityDate,
        flag
    FROM
        `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
    WHERE
        customerid = '4244434'
    ORDER BY
        raw_ingestion_time ASC
),

MixPanel_data AS (
    SELECT
        COALESCE(CAST(mp_user_id AS STRING), CAST(guestid AS STRING)) AS customer_id,
        mp_event_name,
        ctatext,
        cardtitle,
        DATETIME(TIMESTAMP(time), 'Asia/Kolkata') AS time
    FROM
        `abcd-dataplatform-prod.abcd_mixpanel_raw.abcd_mp_master_event`
    WHERE
        COALESCE(CAST(mp_user_id AS STRING), CAST(guestid AS STRING)) = '4244434'
),

joined_data AS (
    SELECT
        anagog.customer_id AS anagog_id,
        mixpanel.customer_id AS mp_id,
        anagog.ActivityDate AS anagog_event_time,
        mixpanel.time AS mp_event_time,
        anagog.PageLastVisit AS anagog_event_name,
        mixpanel.mp_event_name,
        mixpanel.ctatext,
        RANK() OVER (
            PARTITION BY 
                anagog.customer_id, 
                anagog.ActivityDate 
            ORDER BY 
                ABS(TIMESTAMP_DIFF(DATETIME(anagog.ActivityDate), mixpanel.time, SECOND)) ASC
            ) AS rank
    FROM
        Anagog_data AS anagog
    LEFT JOIN
        MixPanel_data AS mixpanel
    ON
        anagog.customer_id = mixpanel.customer_id
        AND (anagog.PageLastVisit = CONCAT(mixpanel.mp_event_name, "_", mixpanel.ctatext))
    QUALIFY rank = 1
)

SELECT
    *
FROM
    joined_data
WHERE
    mp_id IS NOT NULL
"""

mp_hit = client.query(QUERY).to_dataframe()
mp_hit

In [ ]:
mp_hit[['anagog_event_time', 'mp_event_time']]

In [ ]:
%%bigquery test_hit
SELECT
    COALESCE(CAST(mp_user_id AS STRING), CAST(guestid AS STRING)),
    mp_event_name,
    ctatext,
    cardtitle,
    time
FROM
    `abcd-dataplatform-prod.abcd_mixpanel_raw.abcd_mp_master_event`
WHERE
    COALESCE(CAST(mp_user_id AS STRING), CAST(guestid AS STRING)) = '4244434'
    AND mp_event_name = 'hd_mm_section_card_click'

In [ ]:
test_hit.sort_values('time', ascending=False)

In [ ]:
test_hit.mp_event_name.unique()

In [ ]:
test_hit.ctatext.unique()

In [ ]:
test_hit.cardtitle.unique()

In [ ]:
test_hit[test_hit.mp_event_name.isin(['digigold_proceed_now_cta'])]

In [ ]:
%%bigquery anagog_event_names
SELECT
    DISTINCT PageLastVisit
FROM
    `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`

In [ ]:
anagog_event_names

In [ ]:
%%bigquery anagog_event_names
SELECT
    customerid,
    ingestion_time,
    raw_ingestion_time,
    PageLastVisit
FROM
    `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
WHERE
    PageLastVisit = 'digigold_proceed_now_cta'

In [ ]:
anagog_event_names.sort_values('ingestion_time')

In [ ]:
anagog_event_names.sort_values('raw_ingestion_time')

In [ ]:
QUERY = f"""
SELECT
  *
--  column_name,             -- Top-level column name
--  field_path,              -- Full path to the nested field
--  data_type,               -- Data type of the field (ARRAY, STRUCT, STRING, etc.)
--  is_nullable             -- Whether the field is nullable
FROM 
  `region-asia-south1.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE 
  table_catalog = 'abcd-dataplatform-prod'
--  AND table_name = 'users_digitap_sms_data_transformed'
ORDER BY 
  field_path;
"""

test = client.query(QUERY).to_dataframe()
test

In [ ]:
test.table_name.unique()

In [ ]:
QUERY = f"""
SELECT * 
FROM `region-asia-south1.INFORMATION_SCHEMA.SCHEMATA`
WHERE schema_name = 'abcd_digitap_transformed';
"""

test = client.query(QUERY).to_dataframe()
test

In [ ]:
customer_id = pd.read_excel('DigiGold ATS Analysis.xlsx', sheet_name='Raw Data')['customer_id']
print(customer_id)

In [ ]:
QUERY = f"""
SELECT
    DISTINCT customer_id,
    Affluence_Score_Updated AS `Affluence Bucket`,
    SEGMENTATION_CW_SCORE_BUCKET_UPDATED AS `Credit Worthiness Bucket`,
    MOBILE_DEVICE_PRICE AS `Mobile Device Price Bucket`
FROM
    `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`
"""

affluence_maps = client.query(QUERY).to_dataframe()
affluence_maps.info(memory_usage='deep')

In [ ]:
customer_id = customer_id.astype(str)
customer_id

In [ ]:
result = pd.merge(
    left=customer_id,
    right=affluence_maps,
    how='left',
    on='customer_id'
)

In [ ]:
result.head()

In [ ]:
result.to_csv('Affluence Bucket map.csv', index=False)

# Experian Latest stuff?

In [ ]:
%%bigquery test

SELECT
    CUSTOMER_ID,
    COUNT(PINCODE) AS row_count,
    COUNT(DISTINCT PINCODE) AS pincode_count,
    MAX(REPORT_DATE) AS latest_report_date,
    MIN(REPORT_DATE) AS earliest_report_date
FROM
    `abcd-dataplatform.abcd_data_science_app.experian_latest_pincode`
GROUP BY
    CUSTOMER_ID
ORDER BY
    row_count DESC,
    pincode_count DESC

In [ ]:
test.head()

In [ ]:
QUERY = f"""
CREATE OR REPLACE TABLE `abcd-dataplatform.abcd_data_science_app.experian_latest_pincode`
AS (
    SELECT
        CUSTOMER_ID,
        PINCODE,
        SAFE.PARSE_DATE('%d/%m/%Y', DATE_REPORTED) AS REPORT_DATE,
        RANK() OVER (PARTITION BY CUSTOMER_ID ORDER BY SCRUB_DATE DESC, SAFE.PARSE_DATE('%d/%m/%Y', DATE_REPORTED) DESC) AS rank
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ADDRESS_DAILY_BASE`
    QUALIFY rank = 1
)
"""

client.query(QUERY).result()

In [ ]:
QUERY = f"""
SELECT
    *
FROM
    `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ADDRESS_DAILY_BASE`
WHERE
    CUSTOMER_ID = '9012775008'
"""

trial = client.query(QUERY).to_dataframe()
trial

In [ ]:
QUERY = f"""
SELECT
    customer_id AS mobilenumber,
    COUNT(DISTINCT scrub_date) AS scrub_date_count,
    ARRAY_AGG(DISTINCT scrub_date) AS distinct_scrub_dates
FROM
    `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
GROUP BY
    customer_id
HAVING
    scrub_date_count > 1
"""

test = client.query(QUERY).to_dataframe()
test

In [ ]:
trial = test.explode(column='distinct_scrub_dates').reset_index(drop=True)

In [ ]:
trial.distinct_scrub_dates.value_counts()

# Customer Segmentation Columns Check

In [ ]:
QUERY = f"""
WITH customer_segmentation_columns AS (
    SELECT
        column_name,
        data_type
    FROM
        `abcd-dataplatform.abcd_data_science_app.INFORMATION_SCHEMA.COLUMNS`
    WHERE
        table_name = 'CUSTOMER_SEGMENTATION_MASTER_TABLE_ALL_FEATURES'
),

exp_feature_mart_columns AS (
    SELECT
        column_name,
        data_type,
        'experian' AS source
    FROM
        `abcd-dataplatform-prod.abcd_curated_zone_feature_mart.INFORMATION_SCHEMA.COLUMNS`
    WHERE
        table_name = 'EXPERIAN_FEATURE_BASE_DAILY'
),

segmentation_feature_mart_columns AS (
    SELECT
        column_name,
        data_type,
        'segmentation' AS source
    FROM
        `abcd-dataplatform-prod.abcd_curated_zone_feature_mart.INFORMATION_SCHEMA.COLUMNS`
    WHERE
        table_name = 'segment_features'
),

unioned_columns AS (
    SELECT
        *
    FROM (
        SELECT * FROM exp_feature_mart_columns
        UNION ALL
        SELECT * FROM segmentation_feature_mart_columns
    )
)

SELECT
    column_name,
    data_type
FROM
    customer_segmentation_columns
WHERE
    column_name NOT IN (SELECT column_name FROM unioned_columns)
"""

missing_columns = client.query(QUERY).to_dataframe()
missing_columns

In [ ]:
affluence_feature = pd.read_csv('Affluent_columns.csv')

In [ ]:
missing_columns[
    missing_columns.column_name.str.lower().isin(affluence_feature.Feature.str.lower().tolist())
].shape

In [ ]:
QUERY = f"""
SELECT
    column_name,
    data_type,
    'experian' AS source
FROM
    `abcd-dataplatform-prod.abcd_curated_zone_feature_mart.INFORMATION_SCHEMA.COLUMNS`
WHERE
    table_name = 'EXPERIAN_FEATURE_BASE_DAILY'
    AND column_name LIKE '%HFL%'
"""

exp_hl_columns = client.query(QUERY).to_dataframe()
exp_hl_columns

In [ ]:
exp_hl_columns.column_name.tolist()

In [7]:
QUERY = f"""
SELECT
    sms_base.user_id,
    CASE 
        WHEN LENGTH(sms_base.user_id) >= 10 THEN RIGHT(sms_base.user_id, 10)
        ELSE sms_base.user_id
    END AS mobilenumber,
    LENGTH(sms_base.user_id) AS original_mobile_length,
    sms_base.year_month,
    insurance.paid_amount,
    insurance.paid_date,
    insurance.due_amount,
    insurance.due_date,
    insurance.insurance_type,
    insurance.insurance_no,
    insurance.insurance_name
FROM
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS sms_base,
    UNNEST(balances.insurance) AS insurance
WHERE
    insurance.due_date > '2025-02-16' and insurance.due_date < '2025-05-01'
"""

test = client.query(QUERY).to_dataframe()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28583 entries, 0 to 28582
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   user_id                 28583 non-null  object
 1   mobilenumber            28583 non-null  object
 2   original_mobile_length  28583 non-null  Int64 
 3   year_month              28583 non-null  object
 4   paid_amount             28583 non-null  object
 5   paid_date               28583 non-null  object
 6   due_amount              27581 non-null  object
 7   due_date                28583 non-null  object
 8   insurance_type          28583 non-null  object
 9   insurance_no            28583 non-null  object
 10  insurance_name          28583 non-null  object
dtypes: Int64(1), object(10)
memory usage: 2.4+ MB


In [ ]:
test.to_csv('insurance_data.csv', index=False)

In [ ]:
test.insurance_type.value_counts(dropna=False)

In [8]:
test.head()

,user_id,mobilenumber,original_mobile_length,year_month,paid_amount,paid_date,due_amount,due_date,insurance_type,insurance_no,insurance_name
0,9658694839,9658694839,10,2024-06,0.0,,0.0,2025-03-15,LIFE,574043251,LIC
1,108249459567,8249459567,12,2024-04,0.0,,0.0,2025-03-30,LIFE,574895447,LIC
2,108249459567,8249459567,12,2024-04,0.0,,0.0,2025-03-30,LIFE,849743312,LIC
3,109373665136,9373665136,12,2025-01,0.0,,"5,903.21",2025-02-28,LIFE,944475830,LIC
4,109382972103,9382972103,12,2025-01,0.0,,0.0,2025-02-28,LIFE,508700748,LIC


In [9]:
test.original_mobile_length.value_counts()

original_mobile_length
10    27344
12     1239
Name: count, dtype: Int64

In [10]:
test.groupby('user_id').agg({
    'original_mobile_length': 'max'
}).value_counts()

original_mobile_length
10                        21917
12                          917
Name: count, dtype: int64

# Extras

In [4]:
df = client.query(
"""SELECT
    customer_id,
    mobile,
    mobile_device_price
FROM
    `abcd-dataplatform.abcd_data_science_app.CUSTOMER_SEGMENTATION_MASTER_TABLE`"""
).to_dataframe()
df.to_parquet('mobile_device_price_table.parquet', index=False)